# Análise Completa de Principalidade - Dashboard BI0730

Este notebook contém uma análise completa dos dados de principalidade, incluindo:

- Análise de quantidade de contas (PF/PJ) e agências
- Análise BBM vs não-BBM
- Evolução da principalidade nos últimos 3 meses
- Uso de produtos e diferenças de ISA
- Chave PIX e chave forte
- Diferença entre cash-in e cash-out
- Produtos faltantes e análise de categorias
- SOW Sicredi
- Análise temporal de movimentação
- Pré-churn com Machine Learning
- Gráficos e visualizações para apresentação

**Data de Execução**: Agosto 2025

In [ ]:
# Importação de bibliotecas necessárias
import os
import sys
import duckdb
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from datetime import datetime, date, timedelta
import json
import glob
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
import scipy.stats as stats
import pyarrow.parquet as pq  # <- necessário para read_schema

# Configurações
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None

# Configurações do sistema
sys.path.append(rf"C:\Git\BI0730")
from libs.geral.utils import * 

USER = os.getlogin()
ANO = 2025

# Função utilitária simples de merge + coalesce (SEM tratamentos extras)
def merge_and_coalesce(df_left, df_right, on: str, cols_to_merge, how='left'):
    """Merge básico e para cada coluna em cols_to_merge (se existir no right) preenche nulos do left.
    Não cria colunas novas além do necessário e não aplica nenhuma inferência.
    """
    if df_right is None or isinstance(df_right, pd.DataFrame) and df_right.empty:
        print('merge_and_coalesce: df_right vazio -> retorno df_left')
        return df_left
    if not isinstance(cols_to_merge, (list, tuple)):
        cols_to_merge = []
    disponiveis = [c for c in cols_to_merge if c in df_right.columns and c != on]
    if not disponiveis:
        print('merge_and_coalesce: nenhuma coluna disponível para mesclar')
        return df_left
    tmp = df_right[[on] + disponiveis].copy()
    out = df_left.merge(tmp, on=on, how=how, suffixes=('', '_y'))
    for col in disponiveis:
        cy = f'{col}_y'
        if cy in out.columns:
            if col in out.columns:
                out[col] = out[col].where(out[col].notna(), out[cy])
            else:
                out[col] = out[cy]
            out.drop(columns=[cy], inplace=True, errors='ignore')
    return out

# Rutas e configurações baseadas no notebook original
PATH_BASES_PARQUET = rf"C:\Users\{USER}\Sicredi\TimeBI_0730 - Documentos\_BASES\arquivos_parquet"

RUTAS = {
    'inadimplentes': os.path.join(PATH_BASES_PARQUET, 'base_inadimplentes.parquet'),
    'associados_dir': os.path.join(PATH_BASES_PARQUET, 'associados_total_diario'),
    'dashboard': rf"C:\Users\carola_luco\Sicredi\TimeBI_0730 - Documentos\_BASES\arquivos_parquet\associados_totais_tratados_dashboard.parquet",
    'saida_dir': rf'C:\Users\{USER}\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira',
    'saida_dir_giro': rf'C:\Users\{USER}\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira\giro_de_carteira',
    'principalidade': rf"C:\Users\{USER}\Sicredi\TimeBI_0730 - Documentos\_BASES\arquivos_parquet\cia_pcp_indicador_principalidade_historico\2025*.parquet"
}

config = {
    'ano': ANO,
    'pasta_isa_historico': rf"C:\Users\carola_luco\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira\isa_historicos_extracao",
    'filtros_risco_bbm': ["BAIXÍSSIMO", "BAIXO 1", "BAIXO 2", "MÉDIO 1", "MÉDIO 2"],
    'dias_sem_movimentacao': (20, 45)
}

PARAMS = {
    'filtros_risco_bbm': ["BAIXÍSSIMO", "BAIXO 1", "BAIXO 2", "MÉDIO 1", "MÉDIO 2"],
    'dias_sem_movimentacao': (20, 45),
    'principalidade': ['sow']
}

print("✓ Configuração inicial concluída")
print(f"Usuário: {USER}")
print(f"Ano de análise: {ANO}")
print(f"Data de execução: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")

## 1. Carga de Dados Base e Merges

Carregamento dos parquets principais: principalidade histórico, dashboard e ISA histórico.

In [ ]:
# Criar conexão DuckDB e carregar dados de principalidade (SEM QUALQUER NORMALIZAÇÃO / PADRONIZAÇÃO)
con = duckdb.connect()

# ATENÇÃO: Conforme solicitação do usuário TODA normalização automática (zeros à esquerda, renomeios, inferências)
# FOI REMOVIDA. A base é carregada exatamente como armazenada no parquet. Qualquer tratamento de cpf/cnpj
# deverá ser feito manualmente fora deste notebook ou em células específicas que o usuário controlar.

path_principalidade = RUTAS['principalidade']
print(f"Carregando dados de principalidade (bruto): {path_principalidade}")

query_principalidade = f"""
    SELECT *
    FROM read_parquet('{path_principalidade}')
"""

df_principalidade = con.sql(query_principalidade).fetchdf()

print(f"✓ Dados de principalidade carregados (sem transformação): {len(df_principalidade):,} registros")
print(f"  Colunas disponíveis: {len(df_principalidade.columns)}")
if 'ano_mes' in df_principalidade.columns:
    print(f"  Período: {df_principalidade['ano_mes'].min()} até {df_principalidade['ano_mes'].max()}")

print("\nEstrutura (dtypes amostra):")
print(df_principalidade.dtypes.head(12))


In [ ]:
# Carregar dados do dashboard (SEM normalização / sem zfill)
path_dashboard = RUTAS.get('dashboard')

print(f"Carregando dashboard bruto: {path_dashboard}")

# Ler esquema para validar colunas disponíveis
schema_cols = list(pq.read_schema(path_dashboard).names)

# Colunas explicitamente permitidas (adicionados: pix_trans_30d, ult_movimento)
base_cols = [
    'cpf_cnpj','nome_agencia','mc_total_1','gestor','cod_agencia','num_conta',
    'cod_carteira','tipo_pessoa','segmento','segmento_cliente',
    'pix_trans_30d','ult_movimento'  # <- NOVO
]
cols_to_read = [c for c in base_cols if c in schema_cols]

assoc_dash = pd.read_parquet(path_dashboard, columns=cols_to_read)

# ÚNICO rename permitido: segmento_cliente -> segmento (se segmento não existir)
if 'segmento_cliente' in assoc_dash.columns and 'segmento' not in assoc_dash.columns:
    assoc_dash = assoc_dash.rename(columns={'segmento_cliente':'segmento'})

print(f"✓ Dashboard carregado: {len(assoc_dash):,} registros")
print("Colunas:", list(assoc_dash.columns))
# Verificação rápida dos novos campos
for novo_col in ['pix_trans_30d','ult_movimento']:
    if novo_col not in assoc_dash.columns:
        print(f"⚠️  Campo {novo_col} não disponível no dashboard.")
assoc_dash.head(3)

In [ ]:
# Carregar ISA histórico (SEM normalização automática)
isa_dir = config['pasta_isa_historico']
isa_files = sorted(glob.glob(os.path.join(isa_dir, 'isa_historico_analise_*.parquet')))

if isa_files:
    ultimo_isa = isa_files[-1]
    print(f"Carregando ISA histórico (bruto): {ultimo_isa}")
    isa_historico = pd.read_parquet(ultimo_isa)

    col_isa_mes = [c for c in isa_historico.columns if re.match(r'isa_\d{6}$', c)]
    col_isa_mes_sorted = sorted(col_isa_mes)
    ultimas4 = col_isa_mes_sorted[-4:]

    print(f"✓ ISA histórico carregado: {len(isa_historico):,} registros")
    print(f"Colunas: {list(isa_historico.columns)[:20]} ... total={len(isa_historico.columns)}")
    print(f"  Últimas colunas ISA detectadas: {ultimas4}")

    if len(ultimas4) >= 4:
        isa_atual_col = ultimas4[-1]
        tres_anteriores = ultimas4[-4:-1]
    else:
        isa_atual_col = ultimas4[-1] if ultimas4 else None
        tres_anteriores = ultimas4[:-1] if len(ultimas4) > 1 else []

    if tres_anteriores:
        cols_exist = [c for c in tres_anteriores if c in isa_historico.columns]
        isa_historico['isa_media_3m_calc'] = isa_historico[cols_exist].mean(axis=1)
    else:
        isa_historico['isa_media_3m_calc'] = np.nan
else:
    print("⚠️  ISA histórico não encontrado")
    isa_historico = pd.DataFrame()

print("\n✓ Carga bruta concluída (nenhuma padronização aplicada).")

In [ ]:
# Tratamento mínimo solicitado (APENAS rename num_cpf_cnpj -> cpf_cnpj nas bases carregadas)
# Não aplica zfill, strip, nem qualquer outra normalização.

def _rename_num_cpf(df, nome):
    if isinstance(df, pd.DataFrame) and not df.empty and 'num_cpf_cnpj' in df.columns and 'cpf_cnpj' not in df.columns:
        df.rename(columns={'num_cpf_cnpj':'cpf_cnpj'}, inplace=True)
        print(f"✓ {nome}: renomeado 'num_cpf_cnpj' -> 'cpf_cnpj'")
    else:
        print(f"{nome}: nenhum rename necessário")
    return df

bases_tratamento = [
    ('df_principalidade', 'df_principalidade'),
    ('assoc_dash', 'assoc_dash'),
    ('isa_historico', 'isa_historico')
]

for var_name, label in bases_tratamento:
    if var_name in globals():
        globals()[var_name] = _rename_num_cpf(globals()[var_name], label)
    else:
        print(f"{label}: variável inexistente")

print("Tratamento mínimo concluído. Prosseguir com verificação e merges.")

In [ ]:
# Verificação de 'tipo_pessoa' nas bases antes do merge (sem qualquer manipulação)
print("--- Verificação de 'tipo_pessoa' (valores brutos) ---")

bases_info = [
    ('df_principalidade', df_principalidade),
    ('assoc_dash', assoc_dash),
    ('isa_historico', isa_historico if 'isa_historico' in globals() else pd.DataFrame())
]

snapshot_tipo = {}
for nome, df in bases_info:
    if df is not None and not df.empty and 'tipo_pessoa' in df.columns:
        vals = df['tipo_pessoa'].dropna().astype(str).unique()[:10]
        print(f"\n{nome}: valores únicos de tipo_pessoa (primeiros 10):")
        print(vals)
        print(f"Nulos: {df['tipo_pessoa'].isna().sum():,}")
        snapshot_tipo[nome] = df[['cpf_cnpj','tipo_pessoa']].copy() if 'cpf_cnpj' in df.columns else df[['tipo_pessoa']].copy()
    else:
        print(f"\n{nome}: coluna 'tipo_pessoa' ausente ou dataframe vazio (ok se origem não fornece).")
        snapshot_tipo[nome] = pd.DataFrame()

snapshot_assoc_dash = snapshot_tipo.get('assoc_dash')
snapshot_princ = snapshot_tipo.get('df_principalidade')
snapshot_isa = snapshot_tipo.get('isa_historico')
print("\nSnapshot armazenado: snapshot_assoc_dash, snapshot_princ, snapshot_isa.")


In [ ]:
def padroniza_identificadores(df: pd.DataFrame) -> pd.DataFrame:
    """
    Normaliza e padroniza as colunas de um DataFrame, com foco em chaves de junção.
    Regras (EXATAMENTE como solicitado):
      - Renomeia 'num_cpf_cnpj' para 'cpf_cnpj' se existir.
      - Padroniza 'cpf_cnpj' com zeros à esquerda: se len > 11 trata como CNPJ (14), caso contrário CPF (11).
      - Padroniza 'cod_agencia', 'num_conta', 'cod_carteira' apenas com zfill (sem heurística extra).
    Nenhum merge, nenhuma inferência adicional. Apenas retorna o DF tratado.
    """
    if df is None or not isinstance(df, pd.DataFrame) or df.empty:
        return df

    df = df.copy()

    # Renomear coluna principal
    if 'num_cpf_cnpj' in df.columns and 'cpf_cnpj' not in df.columns:
        df = df.rename(columns={'num_cpf_cnpj': 'cpf_cnpj'})

    if 'cpf_cnpj' not in df.columns:
        print("⚠️  Aviso: Coluna 'cpf_cnpj' não encontrada para padronização.")
        return df

    # Padronização cpf_cnpj (sem remover caracteres, somente strip e zfill conforme regra original)
    df['cpf_cnpj'] = (df['cpf_cnpj'].astype(str).str.strip()
                      .apply(lambda x: x.zfill(14) if len(x) > 11 else x.zfill(11)))

    # Demais colunas (apenas se existirem)
    if 'cod_agencia' in df.columns:
        df['cod_agencia'] = df['cod_agencia'].astype(str).str.zfill(2)
    if 'num_conta' in df.columns:
        df['num_conta'] = df['num_conta'].astype(str).str.zfill(6)
    if 'cod_carteira' in df.columns:
        df['cod_carteira'] = np.where(
            df['cod_carteira'].astype(str).str.len() > 3,
            df['cod_carteira'].astype(str).str.zfill(6),
            df['cod_carteira'].astype(str).str.zfill(3)
        )

    return df

print('\n=== Padronizando identificadores das três bases (sem merges) ===')
if 'df_principalidade' in globals():
    df_principalidade = padroniza_identificadores(df_principalidade)
if 'assoc_dash' in globals():
    assoc_dash = padroniza_identificadores(assoc_dash)
if 'isa_historico' in globals():
    isa_historico = padroniza_identificadores(isa_historico)
print('Concluído. Continue com os merges nas células seguintes.')

In [ ]:
assoc_dash = padroniza_identificadores(assoc_dash)
df_principalidade = padroniza_identificadores(df_principalidade)
isa_historico = padroniza_identificadores(isa_historico)

In [ ]:
df_principalidade['porte_padrao'].unique()


In [ ]:
# NOVO MERGE CONTROLADO (usa apenas dados brutos carregados acima)
print('--- Iniciando novo processo de merge controlado ---')

df_base = df_principalidade.copy()

# Merge dashboard
if 'assoc_dash' in globals() and isinstance(assoc_dash, pd.DataFrame) and not assoc_dash.empty:
    if 'cpf_cnpj' not in assoc_dash.columns:
        raise KeyError("Dashboard sem coluna 'cpf_cnpj'.")
    cols_dash_to_merge = [c for c in assoc_dash.columns if c != 'cpf_cnpj']
    df_base = df_base.merge(assoc_dash[['cpf_cnpj'] + cols_dash_to_merge], on='cpf_cnpj', how='left', suffixes=('', '_dash'))
    # Coalesce sem sobrescrever valores já existentes em df_base (df_base é cópia bruta, então só pega do dash se nulo)
    for col in cols_dash_to_merge:
        col_y = f'{col}_dash'
        if col_y in df_base.columns:
            if col not in df_base.columns:
                df_base[col] = df_base[col_y]
            else:
                df_base[col] = df_base[col].where(df_base[col].notna(), df_base[col_y])
            df_base.drop(columns=[col_y], inplace=True)
    print(f"✓ Merge dashboard concluído. Shape: {df_base.shape}")
else:
    print('⚠️ Dashboard não carregado ou vazio')

# Merge ISA (não deve alterar tipo_pessoa)
if 'isa_historico' in globals() and isinstance(isa_historico, pd.DataFrame) and not isa_historico.empty:
    if 'cpf_cnpj' in isa_historico.columns:
        cols_isa_to_merge = [c for c in isa_historico.columns if c != 'cpf_cnpj' and c != 'tipo_pessoa']
        df_base = df_base.merge(isa_historico[['cpf_cnpj'] + cols_isa_to_merge], on='cpf_cnpj', how='left', suffixes=('', '_isa'))
        for col in cols_isa_to_merge:
            col_y = f'{col}_isa'
            if col_y in df_base.columns:
                if col not in df_base.columns:
                    df_base[col] = df_base[col_y]
                else:
                    df_base[col] = df_base[col].where(df_base[col].notna(), df_base[col_y])
                df_base.drop(columns=[col_y], inplace=True)
        print(f"✓ Merge ISA concluído. Shape: {df_base.shape}")
    else:
        print("⚠️ ISA histórico sem coluna 'cpf_cnpj' - merge ignorado.")
else:
    print('⚠️ ISA não carregado ou vazio')




print('Distribuição final tipo_pessoa em df_base:')
if 'tipo_pessoa' in df_base.columns:
    print(df_base['tipo_pessoa'].value_counts(dropna=False))
else:
    print('Coluna tipo_pessoa ausente.')

print('Colunas finais após merges (primeiras 40):')
print(list(df_base.columns)[:40])
print('Total colunas:', len(df_base.columns))

In [ ]:
df_base.query("tipo_pessoa.isna()", engine = 'python')

In [ ]:
# Diagnóstico: onde 'PJ' se perdeu?
# Motivo provável: merge LEFT partindo de df_principalidade elimina cpfs que só existem no dashboard (muitos PJ).
# Esta célula identifica:
# 1. Quantos PJ existem no dashboard bruto.
# 2. Quantos desses PJ aparecem em df_base após o merge.
# 3. Quantos PJ foram perdidos por ausência na principalidade.
# 4. Amostras de cpfs perdidos.
# 5. (Opcional) Reconstrução de df_base usando união de todas as chaves para preservar PJ.



if 'assoc_dash' in globals() and 'tipo_pessoa' in assoc_dash.columns:
    pj_dash_total = (assoc_dash['tipo_pessoa'] == 'PJ').sum()
    pf_dash_total = (assoc_dash['tipo_pessoa'] == 'PF').sum()
    print(f"Dashboard bruto -> PJ: {pj_dash_total:,} | PF: {pf_dash_total:,}")
else:
    print('Dashboard não disponível para diagnóstico.')

if 'snapshot_assoc_dash' in globals() and snapshot_assoc_dash is not None and not snapshot_assoc_dash.empty:
    pj_snapshot = (snapshot_assoc_dash['tipo_pessoa'] == 'PJ').sum()
    print(f"Snapshot assoc_dash -> PJ: {pj_snapshot:,}")

if 'df_base' in globals() and 'tipo_pessoa' in df_base.columns:
    pj_df_base = (df_base['tipo_pessoa'] == 'PJ').sum()
    pf_df_base = (df_base['tipo_pessoa'] == 'PF').sum()
    na_df_base = df_base['tipo_pessoa'].isna().sum()
    print(f"Após merges -> PJ: {pj_df_base:,} | PF: {pf_df_base:,} | NA: {na_df_base:,}")

    # CPFs PJ no dashboard
    pj_cpfs_dash = set(assoc_dash.loc[assoc_dash['tipo_pessoa']=='PJ','cpf_cnpj']) if 'assoc_dash' in globals() else set()
    # CPFs presentes em df_base
    base_cpfs = set(df_base['cpf_cnpj'])
    # PJ preservados
    pj_preservados = pj_cpfs_dash & base_cpfs
    # PJ perdidos
    pj_perdidos = pj_cpfs_dash - base_cpfs
    print(f"PJ preservados (existem em principalidade): {len(pj_preservados):,}")
    print(f"PJ perdidos (não existem em principalidade, por isso não aparecem no LEFT merge): {len(pj_perdidos):,}")
    if pj_perdidos:
        exemplo_perdidos = list(pj_perdidos)[:10]
        print('Exemplos de PJ perdidos:', exemplo_perdidos)
else:
    print('df_base ou tipo_pessoa indisponível para diagnóstico.')

# Opcional: reconstruir df_base com união de chaves para preservar PJ
RECONSTRUIR_UNIAO = False  # coloque True se quiser gerar uma versão alternativa preservando todos os CPFs
if RECONSTRUIR_UNIAO:
    print('\nReconstruindo df_base_all_keys com união de chaves...')
    chaves = set()
    if 'df_principalidade' in globals():
        chaves |= set(df_principalidade.get('cpf_cnpj', []))
    if 'assoc_dash' in globals():
        chaves |= set(assoc_dash.get('cpf_cnpj', []))
    if 'isa_historico' in globals():
        chaves |= set(isa_historico.get('cpf_cnpj', []))
    df_base_all_keys = pd.DataFrame({'cpf_cnpj': list(chaves)})
    # merge incremental
    if 'df_principalidade' in globals():
        df_base_all_keys = df_base_all_keys.merge(df_principalidade, on='cpf_cnpj', how='left')
    if 'assoc_dash' in globals():
        cols_dash_tmp = [c for c in assoc_dash.columns if c != 'cpf_cnpj']
        df_base_all_keys = df_base_all_keys.merge(assoc_dash[['cpf_cnpj'] + cols_dash_tmp], on='cpf_cnpj', how='left', suffixes=('', '_dash2'))
        for c in cols_dash_tmp:
            cy = f'{c}_dash2'
            if cy in df_base_all_keys.columns:
                df_base_all_keys[c] = df_base_all_keys[c].where(df_base_all_keys[c].notna(), df_base_all_keys[cy])
                df_base_all_keys.drop(columns=[cy], inplace=True)
    if 'isa_historico' in globals():
        cols_isa_tmp = [c for c in isa_historico.columns if c != 'cpf_cnpj' and c != 'tipo_pessoa']
        df_base_all_keys = df_base_all_keys.merge(isa_historico[['cpf_cnpj'] + cols_isa_tmp], on='cpf_cnpj', how='left', suffixes=('', '_isa2'))
        for c in cols_isa_tmp:
            cy = f'{c}_isa2'
            if cy in df_base_all_keys.columns:
                df_base_all_keys[c] = df_base_all_keys[c].where(df_base_all_keys[c].notna(), df_base_all_keys[cy])
                df_base_all_keys.drop(columns=[cy], inplace=True)
    # Reforçar tipo_pessoa com dashboard novamente
    if 'assoc_dash' in globals() and 'tipo_pessoa' in assoc_dash.columns:
        tipos_prioritarios2 = (assoc_dash[['cpf_cnpj','tipo_pessoa']]
                               .dropna()
                               .groupby('cpf_cnpj', as_index=False)
                               .agg(tipo_pessoa=lambda s: 'PJ' if 'PJ' in set(s) else ('PF' if 'PF' in set(s) else s.iloc[0])))
        df_base_all_keys.drop(columns=[c for c in ['tipo_pessoa'] if c in df_base_all_keys.columns], inplace=True)
        df_base_all_keys = df_base_all_keys.merge(tipos_prioritarios2, on='cpf_cnpj', how='left')
    print('Distribuição tipo_pessoa (df_base_all_keys):')
    print(df_base_all_keys['tipo_pessoa'].value_counts(dropna=False))
    print('Amostra df_base_all_keys:')
    display(df_base_all_keys.head())
    print('Use df_base_all_keys se quiser incluir PJ que não aparecem na principalidade.')

In [ ]:
df_base['tipo_pessoa'].unique()

## 2. Análise de Quantidade de Contas (PF/PJ) e Agências

Análise da distribuição de contas por tipo de pessoa e agências.

In [ ]:
# Calcula a variação de pontos e identifica quedas
df_base = df_base.sort_values(by=['cpf_cnpj', 'ano_mes'])
df_base['var_pontos'] = df_base.groupby('cpf_cnpj')['pontos_principalidade'].diff().fillna(0)
df_base['queda_flag'] = (df_base['var_pontos'] < 0).astype(int)
df_base['soma_quedas'] = df_base.groupby('cpf_cnpj')['var_pontos'].transform(lambda x: x[x < 0].sum())

df_base[['cpf_cnpj', 'ano_mes', 'pontos_principalidade', 'var_pontos', 'queda_flag', 'soma_quedas']].head()

In [ ]:
# Análise de quantidade de contas por tipo de pessoa e agência
print("📊 ANÁLISE DE QUANTIDADE DE CONTAS POR TIPO DE PESSOA E AGÊNCIA")
print("=" * 60)

if 'df_base' not in globals():
    raise RuntimeError('df_base inexistente: execute as células de carga e merge primeiro.')

# Dados mais recentes (requer ano_mes)
if 'ano_mes' not in df_base.columns:
    raise ValueError("Coluna 'ano_mes' não encontrada em df_base. Verifique a base de principalidade.")

df_atual = df_base[df_base['ano_mes'] == df_base['ano_mes'].max()].copy()

required_cols = ['tipo_pessoa', 'nome_agencia', 'cod_agencia', 'cpf_cnpj']
faltantes = [c for c in required_cols if c not in df_atual.columns]
if faltantes:
    raise ValueError(f"Colunas necessárias ausentes em df_atual: {faltantes}")

# Agrupar por tipo de pessoa e agência
analise_agencia = (df_atual
                   .groupby(['tipo_pessoa', 'nome_agencia', 'cod_agencia'])
                   .agg(
                       associados_unicos=('cpf_cnpj', 'nunique'),
                       registros_totais=('cpf_cnpj', 'count')
                   )
                   .sort_values(by=['tipo_pessoa', 'associados_unicos'], ascending=[True, False])
                   .reset_index())

# Exibir os resultados
print("Análise de Associados e Registros por Tipo de Pessoa e Agência:")
from IPython.display import display
display(analise_agencia)

# Resumo geral por tipo de pessoa
resumo_tipo_pessoa = analise_agencia.groupby('tipo_pessoa').agg(
    total_associados=('associados_unicos', 'sum'),
    total_registros=('registros_totais', 'sum'),
    numero_de_agencias=('nome_agencia', 'count')
).sort_values(by='total_associados', ascending=False)

print("\nResumo Consolidado por Tipo de Pessoa:")
display(resumo_tipo_pessoa)



In [ ]:
# Análise de quantidade de contas por tipo de pessoa
print("📊 ANÁLISE DE QUANTIDADE DE CONTAS E AGÊNCIAS")
print("=" * 60)

# Dados mais recentes
df_atual = df_base[df_base['ano_mes'] == df_base['ano_mes'].max()].copy()

# Total de contas únicas
total_contas = df_atual['cpf_cnpj'].nunique()
print(f"Total de Contas Únicas: {total_contas:,}")

# Distribuição por tipo de pessoa
dist_pessoa = df_atual['tipo_pessoa'].value_counts()
print(f"\nDistribuição por Tipo de Pessoa:")
for tipo, qtd in dist_pessoa.items():
    perc = (qtd / total_contas) * 100
    print(f"  {tipo}: {qtd:,} ({perc:.1f}%)")

# Total de agências
total_agencias = df_atual['cod_agencia'].nunique()
print(f"\nTotal de Agências: {total_agencias:,}")

# Top 10 agências com mais contas
top_agencias = df_atual['cod_agencia'].value_counts().head(10)
print(f"\nTop 10 Agências (por número de contas):")
for i, (agencia, qtd) in enumerate(top_agencias.items(), 1):
    perc = (qtd / total_contas) * 100
    print(f"  {i:2d}. Agência {agencia}: {qtd:,} contas ({perc:.1f}%)")

# Distribuição PF/PJ por agência (top 5)
print(f"\nDistribuição PF/PJ nas Top 5 Agências:")
top5_agencias = top_agencias.head(5).index

for agencia in top5_agencias:
    df_agencia = df_atual[df_atual['cod_agencia'] == agencia]
    dist_agencia = df_agencia['tipo_pessoa'].value_counts()
    total_agencia = len(df_agencia)
    
    pf_count = dist_agencia.get('PF', 0)
    pj_count = dist_agencia.get('PJ', 0)
    
    print(f"  Agência {agencia}: PF={pf_count} ({pf_count/total_agencia*100:.1f}%) | PJ={pj_count} ({pj_count/total_agencia*100:.1f}%)")

# Resumo consolidado
resumo_contas = {
    'total_contas': total_contas,
    'total_pf': dist_pessoa.get('PF', 0),
    'total_pj': dist_pessoa.get('PJ', 0),
    'total_agencias': total_agencias,
    'perc_pf': (dist_pessoa.get('PF', 0) / total_contas) * 100,
    'perc_pj': (dist_pessoa.get('PJ', 0) / total_contas) * 100
}

print(f"\n📋 RESUMO EXECUTIVO - CONTAS E AGÊNCIAS")
print(f"Total: {resumo_contas['total_contas']:,} contas em {resumo_contas['total_agencias']:,} agências")
print(f"PF: {resumo_contas['total_pf']:,} ({resumo_contas['perc_pf']:.1f}%)")
print(f"PJ: {resumo_contas['total_pj']:,} ({resumo_contas['perc_pj']:.1f}%)")

# Salvar resumo para uso posterior
globals()['resumo_contas'] = resumo_contas

## 3. Análise BBM e Fora de Filtro

Análise da distribuição dos associados dentro e fora dos filtros de risco BBM.

In [ ]:
# Análise BBM vs Fora de Filtro
print("📊 ANÁLISE BBM E FORA DE FILTRO")
print("=" * 50)

# Classificar por BBM
riscos_bbm = PARAMS['filtros_risco_bbm']
print(f"Filtros BBM considerados: {riscos_bbm}")

# Normalizar nível de risco
df_atual['nivel_risco_norm'] = df_atual['nivel_risco'].astype(str).str.upper().str.strip()

# Classificar BBM vs Não-BBM
df_atual['classificacao_bbm'] = np.where(
    df_atual['nivel_risco_norm'].isin([r.upper() for r in riscos_bbm]),
    'BBM',
    'Fora_BBM'
)

# Contagem por classificação BBM
dist_bbm = df_atual['classificacao_bbm'].value_counts()
total_analisados = len(df_atual)

print(f"\nDistribuição por Classificação BBM:")
for classif, qtd in dist_bbm.items():
    perc = (qtd / total_analisados) * 100
    print(f"  {classif}: {qtd:,} ({perc:.1f}%)")

# Análise detalhada por nível de risco
print(f"\nDetalhamento por Nível de Risco:")
dist_risco = df_atual['nivel_risco_norm'].value_counts()
for risco, qtd in dist_risco.head(10).items():
    perc = (qtd / total_analisados) * 100
    status = "✓ BBM" if risco in [r.upper() for r in riscos_bbm] else "✗ Fora BBM"
    print(f"  {risco}: {qtd:,} ({perc:.1f}%) - {status}")

# Cruzamento BBM x Tipo de Pessoa
print(f"\nCruzamento BBM x Tipo de Pessoa:")
crosstab_bbm_pessoa = pd.crosstab(df_atual['classificacao_bbm'], df_atual['tipo_pessoa'], margins=True)
print(crosstab_bbm_pessoa)

# Percentuais por tipo de pessoa
print(f"\nPercentuais BBM por Tipo de Pessoa:")
for pessoa in ['PF', 'PJ']:
    df_pessoa = df_atual[df_atual['tipo_pessoa'] == pessoa]
    if not df_pessoa.empty:
        bbm_count = (df_pessoa['classificacao_bbm'] == 'BBM').sum()
        total_pessoa = len(df_pessoa)
        perc_bbm = (bbm_count / total_pessoa) * 100
        print(f"  {pessoa}: {bbm_count:,}/{total_pessoa:,} são BBM ({perc_bbm:.1f}%)")

# Análise por faixa de principalidade
if 'faixa_categoria' in df_atual.columns:
    print(f"\nBBM por Faixa de Principalidade:")
    crosstab_bbm_faixa = pd.crosstab(df_atual['faixa_categoria'], df_atual['classificacao_bbm'], margins=True)
    print(crosstab_bbm_faixa)

# Resumo executivo BBM
bbm_count = dist_bbm.get('BBM', 0)
fora_bbm_count = dist_bbm.get('Fora_BBM', 0)

resumo_bbm = {
    'total_analisados': total_analisados,
    'bbm_count': bbm_count,
    'fora_bbm_count': fora_bbm_count,
    'perc_bbm': (bbm_count / total_analisados) * 100,
    'perc_fora_bbm': (fora_bbm_count / total_analisados) * 100
}

print(f"\n📋 RESUMO EXECUTIVO - BBM")
print(f"Dentro do filtro BBM: {resumo_bbm['bbm_count']:,} ({resumo_bbm['perc_bbm']:.1f}%)")
print(f"Fora do filtro BBM: {resumo_bbm['fora_bbm_count']:,} ({resumo_bbm['perc_fora_bbm']:.1f}%)")

globals()['resumo_bbm'] = resumo_bbm

## 4. Análise de Principalidade: Subida e Bajada Últimos 3 Meses

Análise da evolução da principalidade: contas que subiram e desceram nos últimos 3 meses.

In [ ]:
# Análise de Evolução da Principalidade
print("📊 ANÁLISE DE EVOLUÇÃO DA PRINCIPALIDADE - ÚLTIMOS 3 MESES")
print("=" * 70)

# Pegar últimos 3 meses disponíveis
meses_disponiveis = sorted(df_base['ano_mes'].unique())
ultimos_3_meses = meses_disponiveis[-3:] if len(meses_disponiveis) >= 3 else meses_disponiveis

print(f"Períodos analisados: {ultimos_3_meses}")

# Filtrar dados dos últimos 3 meses
df_3meses = df_base[df_base['ano_mes'].isin(ultimos_3_meses)].copy()

# Análise por associado nos últimos 3 meses
evolucao_princ = df_3meses.groupby('cpf_cnpj').agg({
    'pontos_principalidade': ['first', 'last', 'min', 'max', 'std'],
    'var_pontos': 'sum',
    'queda_flag': 'sum',
    'soma_quedas': 'max',
    'tipo_pessoa': 'first',
    'faixa_categoria': 'last'
}).round(2)

# Flatten column names
evolucao_princ.columns = ['_'.join(col).strip() for col in evolucao_princ.columns]
evolucao_princ = evolucao_princ.reset_index()

# Calcular variação total
evolucao_princ['var_total'] = evolucao_princ['pontos_principalidade_last'] - evolucao_princ['pontos_principalidade_first']

# Classificar evolução
def classificar_evolucao(row):
    if row['var_total'] > 0:
        return 'SUBIU'
    elif row['var_total'] < 0:
        return 'DESCEU'
    else:
        return 'ESTÁVEL'

evolucao_princ['classificacao_evolucao'] = evolucao_princ.apply(classificar_evolucao, axis=1)

# Análise geral de evolução
print(f"\nRESULTADOS GERAIS:")
dist_evolucao = evolucao_princ['classificacao_evolucao'].value_counts()
total_associados = len(evolucao_princ)

for classif, qtd in dist_evolucao.items():
    perc = (qtd / total_associados) * 100
    print(f"  {classif}: {qtd:,} associados ({perc:.1f}%)")

# Estatísticas de variação
print(f"\nESTATÍSTICAS DE VARIAÇÃO DOS PONTOS:")
print(f"  Variação média: {evolucao_princ['var_total'].mean():.2f} pontos")
print(f"  Variação mediana: {evolucao_princ['var_total'].median():.2f} pontos")
print(f"  Maior subida: {evolucao_princ['var_total'].max():.2f} pontos")
print(f"  Maior queda: {evolucao_princ['var_total'].min():.2f} pontos")
print(f"  Desvio padrão: {evolucao_princ['var_total'].std():.2f} pontos")

# Evolução por tipo de pessoa
print(f"\nEVOLUÇÃO POR TIPO DE PESSOA:")
for pessoa in ['PF', 'PJ']:
    df_pessoa = evolucao_princ[evolucao_princ['tipo_pessoa_first'] == pessoa]
    if not df_pessoa.empty:
        dist_pessoa = df_pessoa['classificacao_evolucao'].value_counts()
        total_pessoa = len(df_pessoa)
        
        print(f"\n  {pessoa} (Total: {total_pessoa:,}):")
        for classif, qtd in dist_pessoa.items():
            perc = (qtd / total_pessoa) * 100
            print(f"    {classif}: {qtd:,} ({perc:.1f}%)")

# Top 10 maiores subidas
print(f"\nTOP 10 MAIORES SUBIDAS:")
top_subidas = evolucao_princ.nlargest(10, 'var_total')
for i, row in enumerate(top_subidas.itertuples(), 1):
    print(f"  {i:2d}. CPF: {row.cpf_cnpj[-4:]}... | Subida: +{row.var_total:.1f} pontos | {row.tipo_pessoa_first}")

# Top 10 maiores quedas  
print(f"\nTOP 10 MAIORES QUEDAS:")
top_quedas = evolucao_princ.nsmallest(10, 'var_total')
for i, row in enumerate(top_quedas.itertuples(), 1):
    print(f"  {i:2d}. CPF: {row.cpf_cnpj[-4:]}... | Queda: {row.var_total:.1f} pontos | {row.tipo_pessoa_first}")

# Associados com 3 quedas consecutivas (conforme query original)
associados_3_quedas = evolucao_princ[evolucao_princ['soma_quedas_max'] >= 3]
print(f"\nASSOCIADOS COM 3 QUEDAS CONSECUTIVAS: {len(associados_3_quedas):,}")

if len(associados_3_quedas) > 0:
    print(f"  PF: {(associados_3_quedas['tipo_pessoa_first'] == 'PF').sum():,}")
    print(f"  PJ: {(associados_3_quedas['tipo_pessoa_first'] == 'PJ').sum():,}")
    
    # Exemplos de associados com 3 quedas
    print(f"\n  Exemplos (5 primeiros):")
    for i, row in enumerate(associados_3_quedas.head().itertuples(), 1):
        print(f"    {i}. CPF: {row.cpf_cnpj[-4:]}... | Var Total: {row.var_total:.1f} | Quedas: {row.soma_quedas_max}")

# Resumo executivo
resumo_evolucao = {
    'total_analisados': total_associados,
    'subiu': dist_evolucao.get('SUBIU', 0),
    'desceu': dist_evolucao.get('DESCEU', 0), 
    'estavel': dist_evolucao.get('ESTÁVEL', 0),
    'tres_quedas': len(associados_3_quedas),
    'var_media': evolucao_princ['var_total'].mean()
}

print(f"\n📋 RESUMO EXECUTIVO - EVOLUÇÃO PRINCIPALIDADE")
print(f"Analisados: {resumo_evolucao['total_analisados']:,} associados")
print(f"Subiram: {resumo_evolucao['subiu']:,} ({resumo_evolucao['subiu']/total_associados*100:.1f}%)")
print(f"Desceram: {resumo_evolucao['desceu']:,} ({resumo_evolucao['desceu']/total_associados*100:.1f}%)")
print(f"3 quedas consecutivas: {resumo_evolucao['tres_quedas']:,}")

globals()['resumo_evolucao'] = resumo_evolucao
globals()['evolucao_princ'] = evolucao_princ

# Define as colunas específicas do 'assoc_dash' que queremos mesclar.
# Adicionados: 'pix_trans_30d', 'ult_movimento'
cols_dash_to_merge = [
    'tipo_pessoa', 'segmento_cliente', 'perfil_investidor',
    'data_abertura', 'data_encerramento', 'status_conta', 'nome_agencia',
    'pix_trans_30d', 'ult_movimento'  # <- NOVOS CAMPOS
]

# Realiza o merge usando a função atualizada
df_base = merge_and_coalesce(
    df_left=df_principalidade,
    df_right=assoc_dash,
    on='cpf_cnpj',
    cols_to_merge=cols_dash_to_merge,
    how='left'
)

# Verificação de valores nulos em 'tipo_pessoa' após o merge
if 'tipo_pessoa' in df_base.columns and df_base['tipo_pessoa'].isnull().any():
    print("\nAtenção: Existem registros com 'tipo_pessoa' nulo após o merge com assoc_dash.")
    print("Isso indica que alguns CPFs/CNPJs de 'df_principalidade' não foram encontrados em 'assoc_dash'.")

# Garantir conversão de ult_movimento para data (para uso posterior)
if 'ult_movimento' in df_base.columns:
    df_base['ult_movimento'] = pd.to_datetime(df_base['ult_movimento'], errors='coerce')

df_base.info()

## 6. Diferença de ISA Últimos 3 Meses

Análise das variações do ISA nos últimos 3 meses e insights.

In [ ]:
# Análise de Diferença de ISA Últimos 3 Meses
print("📊 ANÁLISE DE DIFERENÇA DE ISA - ÚLTIMOS 3 MESES")
print("=" * 60)

# Verificar disponibilidade de dados ISA
if 'isa_media' in df_base.columns or 'isa_media_3m_calc' in df_base.columns:
    
    # Usar dados ISA disponíveis
    df_isa = df_base.copy()
    
    # Preparar análise ISA
    if 'isa_media_3m_calc' in df_isa.columns:
        print("Usando ISA média 3M calculada")
        df_isa['isa_analise'] = df_isa['isa_media_3m_calc']
    elif 'isa_media' in df_isa.columns:
        print("Usando ISA média disponível")
        df_isa['isa_analise'] = df_isa['isa_media']
    else:
        print("⚠️  Dados de ISA não disponíveis para análise completa")
        df_isa['isa_analise'] = np.nan
    
    # Filtrar dados com ISA válido
    df_isa_valido = df_isa[df_isa['isa_analise'].notna()].copy()
    
    if len(df_isa_valido) > 0:
        print(f"Registros com ISA válido: {len(df_isa_valido):,}")
        
        # Estatísticas gerais de ISA
        print(f"\nESTATÍSTICAS GERAIS DE ISA:")
        isa_stats = df_isa_valido['isa_analise'].describe()
        print(f"  Média: {isa_stats['mean']:.2f}")
        print(f"  Mediana: {isa_stats['50%']:.2f}")
        print(f"  Mínimo: {isa_stats['min']:.2f}")
        print(f"  Máximo: {isa_stats['max']:.2f}")
        print(f"  Desvio Padrão: {isa_stats['std']:.2f}")
        
        # Distribuição por faixas de ISA
        df_isa_valido['faixa_isa'] = pd.cut(
            df_isa_valido['isa_analise'], 
            bins=[0, 2, 4, 6, 8, float('inf')], 
            labels=['0-2', '2-4', '4-6', '6-8', '8+']
        )
        
        print(f"\nDISTRIBUIÇÃO POR FAIXAS DE ISA:")
        dist_isa = df_isa_valido['faixa_isa'].value_counts().sort_index()
        total_isa = len(df_isa_valido)
        
        for faixa, qtd in dist_isa.items():
            perc = (qtd / total_isa) * 100
            print(f"  {faixa}: {qtd:,} ({perc:.1f}%)")
        
        # ISA por tipo de pessoa
        print(f"\nISA MÉDIO POR TIPO DE PESSOA:")
        isa_por_pessoa = df_isa_valido.groupby('tipo_pessoa')['isa_analise'].agg(['mean', 'count', 'std']).round(2)
        
        for pessoa in isa_por_pessoa.index:
            media = isa_por_pessoa.loc[pessoa, 'mean']
            count = isa_por_pessoa.loc[pessoa, 'count']
            std = isa_por_pessoa.loc[pessoa, 'std']
            print(f"  {pessoa}: {media:.2f} (n={count:,}, std={std:.2f})")
        
        # ISA vs Principalidade
        if 'pontos_principalidade' in df_isa_valido.columns:
            print(f"\nCORRELAÇÃO ISA vs PRINCIPALIDADE:")
            correlacao = df_isa_valido['isa_analise'].corr(df_isa_valido['pontos_principalidade'])
            print(f"  Correlação: {correlacao:.3f}")
            
            # Análise por quartis de ISA
            df_isa_valido['quartil_isa'] = pd.qcut(
                df_isa_valido['isa_analise'], 
                q=4, 
                labels=['Q1_Baixo', 'Q2_MedioBaixo', 'Q3_MedioAlto', 'Q4_Alto']
            )
            
            princ_por_quartil = df_isa_valido.groupby('quartil_isa')['pontos_principalidade'].agg(['mean', 'count']).round(1)
            
            print(f"\n  PRINCIPALIDADE MÉDIA POR QUARTIL DE ISA:")
            for quartil in princ_por_quartil.index:
                media_princ = princ_por_quartil.loc[quartil, 'mean']
                count = princ_por_quartil.loc[quartil, 'count']
                print(f"    {quartil}: {media_princ:.1f} pontos (n={count:,})")
        
        # Top/Bottom ISA
        print(f"\nTOP 10 MAIORES ISA:")
        top_isa = df_isa_valido.nlargest(10, 'isa_analise')[['cpf_cnpj', 'isa_analise', 'tipo_pessoa', 'pontos_principalidade']]
        for i, row in enumerate(top_isa.itertuples(), 1):
            cpf_mask = row.cpf_cnpj[-4:] if len(str(row.cpf_cnpj)) >= 4 else str(row.cpf_cnpj)
            print(f"  {i:2d}. CPF: ...{cpf_mask} | ISA: {row.isa_analise:.2f} | {row.tipo_pessoa} | Princ: {row.pontos_principalidade:.1f}")
        
        print(f"\nTOP 10 MENORES ISA (acima de 0):")
        bottom_isa = df_isa_valido[df_isa_valido['isa_analise'] > 0].nsmallest(10, 'isa_analise')[['cpf_cnpj', 'isa_analise', 'tipo_pessoa', 'pontos_principalidade']]
        for i, row in enumerate(bottom_isa.itertuples(), 1):
            cpf_mask = row.cpf_cnpj[-4:] if len(str(row.cpf_cnpj)) >= 4 else str(row.cpf_cnpj)
            print(f"  {i:2d}. CPF: ...{cpf_mask} | ISA: {row.isa_analise:.2f} | {row.tipo_pessoa} | Princ: {row.pontos_principalidade:.1f}")
        
        # Insights ISA
        print(f"\n💡 INSIGHTS ISA:")
        
        # ISA alto com baixa principalidade (oportunidades)
        isa_alto_princ_baixo = df_isa_valido[
            (df_isa_valido['isa_analise'] >= 6) & 
            (df_isa_valido['pontos_principalidade'] < df_isa_valido['pontos_principalidade'].median())
        ]
        
        if len(isa_alto_princ_baixo) > 0:
            print(f"  • {len(isa_alto_princ_alto):,} associados com ISA alto (≥6) mas principalidade baixa")
            print(f"    Oportunidade de cross-selling!")
        
        # ISA baixo com alta principalidade (risco)
        isa_baixo_princ_alto = df_isa_valido[
            (df_isa_valido['isa_analise'] <= 2) & 
            (df_isa_valido['pontos_principalidade'] > df_isa_valido['pontos_principalidade'].quantile(0.75))
        ]
        
        if len(isa_baixo_princ_alto) > 0:
            print(f"  • {len(isa_baixo_princ_alto):,} associados com ISA baixo (≤2) mas principalidade alta")
            print(f"    Possível risco de churn!")
        
        # Concentração ISA por faixa
        faixa_2_4 = (dist_isa.get('2-4', 0) / total_isa) * 100
        if faixa_2_4 > 30:
            print(f"  • {faixa_2_4:.1f}% dos associados na faixa ISA 2-4 (faixa crítica)")
        
        # Resumo ISA
        resumo_isa = {
            'total_com_isa': len(df_isa_valido),
            'isa_medio': df_isa_valido['isa_analise'].mean(),
            'isa_mediano': df_isa_valido['isa_analise'].median(),
            'acima_6': (df_isa_valido['isa_analise'] >= 6).sum(),
            'abaixo_2': (df_isa_valido['isa_analise'] <= 2).sum(),
            'correlacao_princ': correlacao if 'correlacao' in locals() else 0
        }
        
        print(f"\n📋 RESUMO EXECUTIVO - ISA")
        print(f"ISA médio: {resumo_isa['isa_medio']:.2f}")
        print(f"Acima de 6: {resumo_isa['acima_6']:,} ({resumo_isa['acima_6']/total_isa*100:.1f}%)")
        print(f"Abaixo de 2: {resumo_isa['abaixo_2']:,} ({resumo_isa['abaixo_2']/total_isa*100:.1f}%)")
        
        globals()['resumo_isa'] = resumo_isa
        globals()['df_isa_analise'] = df_isa_valido
        
    else:
        print("⚠️  Nenhum registro com ISA válido encontrado")
        resumo_isa = {'total_com_isa': 0, 'isa_medio': 0}
        
else:
    print("⚠️  Colunas de ISA não encontradas nos dados")
    resumo_isa = {'total_com_isa': 0, 'isa_medio': 0}

# Colunas do 'isa_historico' para fazer o merge
cols_isa_to_merge = ['tipo_pessoa', 'segmento']

# Realiza o merge usando a função atualizada
df_base = merge_and_coalesce(
    df_left=df_base,
    df_right=isa_historico,
    on='cpf_cnpj',
    cols_to_merge=cols_isa_to_merge,
    how='left'
)

# Verificação de valores nulos em 'tipo_pessoa' após o merge final
if df_base['tipo_pessoa'].isnull().any():
    print("\nAtenção: Existem registros com 'tipo_pessoa' nulo após o merge final.")
    print("Esses registros não foram encontrados nem em 'assoc_dash' nem em 'isa_historico'.")

df_base.info()

## 7. Chave PIX e Chave Forte Últimos 3 Períodos

Análise do uso de chave PIX e chave forte nos últimos 3 períodos.

In [ ]:
# Análise de Chave PIX e Chave Forte Últimos 3 Períodos
print("📊 ANÁLISE CHAVE PIX E CHAVE FORTE - ÚLTIMOS 3 PERÍODOS")
print("=" * 70)

# Campos PIX disponíveis
campos_pix = {
    'PIX Cadastrado': 'cad_pix',
    'PIX Ativo': 'cad_pix_ativo',
    'Transações PIX 30d': 'pix_trans_30d'
}

# Dados dos últimos 3 períodos
df_pix_3m = df_base[df_base['ano_mes'].isin(ultimos_3_meses)].copy()

print(f"Períodos analisados: {ultimos_3_meses}")
print(f"Total de registros: {len(df_pix_3m):,}")

# ------------------------------------------------------------------
# Normalização de campos PIX para evitar erro:
# Algumas bases usam 'S'/'N', 'SIM'/'NAO' ou strings numéricas.
# Convertemos para numérico (0/1) ou 0 quando não reconhecido.
# ------------------------------------------------------------------
mapeamentos_binarios = {
    'S': 1, 'N': 0,
    'SIM': 1, 'NAO': 0, 'NÃO': 0,
    'Y': 1, 'NOK': 0, 'OK': 1,
    True: 1, False: 0
}

for col in ['cad_pix_ativo', 'pix_trans_30d']:
    if col in df_pix_3m.columns:
        # Substitui valores mapeáveis
        df_pix_3m[col] = df_pix_3m[col].replace(mapeamentos_binarios)
        # Converte strings numéricas; valores não convertíveis viram NaN e depois 0
        df_pix_3m[col] = pd.to_numeric(df_pix_3m[col], errors='coerce').fillna(0)
        # Para coluna binária, força inteiro 0/1
        if col == 'cad_pix_ativo':
            df_pix_3m[col] = (df_pix_3m[col] > 0).astype(int)

# Garantir mesma limpeza no último período (usado mais abaixo)
df_ultimo_periodo = df_pix_3m[df_pix_3m['ano_mes'] == ultimos_3_meses[-1]].copy()
if 'cad_pix_ativo' in df_ultimo_periodo.columns:
    df_ultimo_periodo['cad_pix_ativo'] = df_ultimo_periodo['cad_pix_ativo'].replace(mapeamentos_binarios)
    df_ultimo_periodo['cad_pix_ativo'] = pd.to_numeric(df_ultimo_periodo['cad_pix_ativo'], errors='coerce').fillna(0)
    df_ultimo_periodo['cad_pix_ativo'] = (df_ultimo_periodo['cad_pix_ativo'] > 0).astype(int)
if 'pix_trans_30d' in df_ultimo_periodo.columns:
    df_ultimo_periodo['pix_trans_30d'] = df_ultimo_periodo['pix_trans_30d'].replace(mapeamentos_binarios)
    df_ultimo_periodo['pix_trans_30d'] = pd.to_numeric(df_ultimo_periodo['pix_trans_30d'], errors='coerce').fillna(0)

# Análise evolutiva PIX por período
print(f"\nEVOLUÇÃO PIX POR PERÍODO:")
print(f"{'Período':<10} {'PIX Ativo':<12} {'% Total':<10} {'Transações':<12} {'Variação':<12}")
print("-" * 70)

evolucao_pix = []
for i, periodo in enumerate(ultimos_3_meses):
    df_periodo = df_pix_3m[df_pix_3m['ano_mes'] == periodo]
    total_periodo = len(df_periodo)
    
    # PIX Ativo
    if 'cad_pix_ativo' in df_periodo.columns:
        pix_ativo = df_periodo['cad_pix_ativo'].sum()
        perc_pix_ativo = (pix_ativo / total_periodo) * 100 if total_periodo > 0 else 0
    else:
        pix_ativo = 0
        perc_pix_ativo = 0
    
    # Transações PIX
    if 'pix_trans_30d' in df_periodo.columns:
        pix_transacoes = df_periodo['pix_trans_30d'].sum()
    else:
        pix_transacoes = 0
    
    # Variação em relação ao período anterior
    if i > 0:
        var_pix = pix_ativo - evolucao_pix[-1]['pix_ativo']
        var_perc = ((pix_ativo / evolucao_pix[-1]['pix_ativo']) - 1) * 100 if evolucao_pix[-1]['pix_ativo'] > 0 else 0
        variacao_str = f"{var_pix:+,} ({var_perc:+.1f}%)"
    else:
        variacao_str = "Base"
    
    
    evolucao_pix.append({
        'periodo': periodo,
        'pix_ativo': pix_ativo,
        'perc_pix_ativo': perc_pix_ativo,
        'pix_transacoes': pix_transacoes,
        'total_periodo': total_periodo
    })

# Análise PIX por tipo de pessoa
print(f"\nPIX POR TIPO DE PESSOA (último período):")
df_ultimo_periodo = df_pix_3m[df_pix_3m['ano_mes'] == ultimos_3_meses[-1]]

for pessoa in ['PF', 'PJ']:
    df_pessoa = df_ultimo_periodo[df_ultimo_periodo['tipo_pessoa'] == pessoa]
    if not df_pessoa.empty:
        total_pessoa = len(df_pessoa)
        
        # PIX Ativo
        if 'cad_pix_ativo' in df_pessoa.columns:
            pix_ativo_pessoa = df_pessoa['cad_pix_ativo'].sum()
            perc_pix_pessoa = (pix_ativo_pessoa / total_pessoa) * 100
        else:
            pix_ativo_pessoa = 0
            perc_pix_pessoa = 0
        
        # Transações médias
        if 'pix_trans_30d' in df_pessoa.columns:
            # Se existir coluna de PIX ativo, calcula média apenas para ativos
            if 'cad_pix_ativo' in df_pessoa.columns:
                ativos_mask = df_pessoa['cad_pix_ativo'] == 1
                if ativos_mask.any():
                    trans_media = df_pessoa.loc[ativos_mask, 'pix_trans_30d'].mean()
                else:
                    trans_media = 0
            else:
                trans_media = df_pessoa['pix_trans_30d'].mean()
            if pd.isna(trans_media):
                trans_media = 0
        else:
            trans_media = 0
        
        print(f"  {pessoa}: {pix_ativo_pessoa:,}/{total_pessoa:,} ({perc_pix_pessoa:.1f}%) | Trans/mês: {trans_media:.1f}")

# Chaves fortes (análise baseada em campos disponíveis)
print(f"\nANÁLISE DE CHAVES FORTES:")

# Assumindo que chaves fortes são: CPF, CNPJ, telefone, email
# Como não temos detalhamento, usamos PIX ativo como proxy para chaves fortes
if 'cad_pix_ativo' in df_ultimo_periodo.columns:
    chaves_fortes = df_ultimo_periodo['cad_pix_ativo'].sum()
    total_ultimo = len(df_ultimo_periodo)
    perc_fortes = (chaves_fortes / total_ultimo) * 100
    
    print(f"  Associados com PIX ativo (proxy chaves fortes): {chaves_fortes:,} ({perc_fortes:.1f}%)")
    
    # Por segmento
    for pessoa in ['PF', 'PJ']:
        df_pessoa = df_ultimo_periodo[df_ultimo_periodo['tipo_pessoa'] == pessoa]
        if not df_pessoa.empty:
            fortes_pessoa = df_pessoa['cad_pix_ativo'].sum()
            total_pessoa = len(df_pessoa)
            perc_pessoa = (fortes_pessoa / total_pessoa) * 100
            print(f"    {pessoa}: {fortes_pessoa:,}/{total_pessoa:,} ({perc_pessoa:.1f}%)")

# Análise de intensidade de uso PIX
if 'pix_trans_30d' in df_ultimo_periodo.columns:
    df_pix_users = df_ultimo_periodo[df_ultimo_periodo['cad_pix_ativo'] == 1].copy()
    
    if not df_pix_users.empty:
        print(f"\nINTENSIDADE DE USO PIX (usuários ativos):")
        
        # Classificar por intensidade
        df_pix_users['intensidade_pix'] = pd.cut(
            df_pix_users['pix_trans_30d'],
            bins=[0, 1, 5, 15, float('inf')],
            labels=['Muito_Baixo', 'Baixo', 'Médio', 'Alto']
        )
        
        dist_intensidade = df_pix_users['intensidade_pix'].value_counts()
        total_pix_users = len(df_pix_users)
        
        print(f"  Total usuários PIX: {total_pix_users:,}")
        for intensidade, qtd in dist_intensidade.items():
            perc = (qtd / total_pix_users) * 100
            print(f"    {intensidade}: {qtd:,} ({perc:.1f}%)")
        
        # Média de transações
        media_trans = df_pix_users['pix_trans_30d'].mean()
        mediana_trans = df_pix_users['pix_trans_30d'].median()
        print(f"  Média transações/mês: {media_trans:.1f}")
        print(f"  Mediana transações/mês: {mediana_trans:.1f}")

# PIX vs Principalidade
if 'pontos_principalidade' in df_ultimo_periodo.columns and 'cad_pix_ativo' in df_ultimo_periodo.columns:
    print(f"\nPIX vs PRINCIPALIDADE:")
    
    # Média de principalidade com/sem PIX
    princ_com_pix = df_ultimo_periodo[df_ultimo_periodo['cad_pix_ativo'] == 1]['pontos_principalidade'].mean()
    princ_sem_pix = df_ultimo_periodo[df_ultimo_periodo['cad_pix_ativo'] == 0]['pontos_principalidade'].mean()
    
    print(f"  Com PIX: {princ_com_pix:.1f} pontos de principalidade")
    print(f"  Sem PIX: {princ_sem_pix:.1f} pontos de principalidade")
    print(f"  Diferença: {princ_com_pix - princ_sem_pix:+.1f} pontos")

# Resumo PIX
if evolucao_pix:
    primeiro_periodo = evolucao_pix[0]
    ultimo_periodo = evolucao_pix[-1]
    
    crescimento_pix = ultimo_periodo['pix_ativo'] - primeiro_periodo['pix_ativo']
    crescimento_perc = ((ultimo_periodo['pix_ativo'] / primeiro_periodo['pix_ativo']) - 1) * 100 if primeiro_periodo['pix_ativo'] > 0 else 0
    
    resumo_pix = {
        'pix_ativo_atual': ultimo_periodo['pix_ativo'],
        'perc_penetracao': ultimo_periodo['perc_pix_ativo'],
        'crescimento_absoluto': crescimento_pix,
        'crescimento_percentual': crescimento_perc,
        'transacoes_totais': ultimo_periodo['pix_transacoes']
    }
    
    print(f"\n📋 RESUMO EXECUTIVO - PIX")
    print(f"PIX ativo atual: {resumo_pix['pix_ativo_atual']:,} ({resumo_pix['perc_penetracao']:.1f}%)")
    print(f"Crescimento 3M: {resumo_pix['crescimento_absoluto']:+,} ({resumo_pix['crescimento_percentual']:+.1f}%)")
    print(f"Transações totais: {resumo_pix['transacoes_totais']:,}")
    
    globals()['resumo_pix'] = resumo_pix
    globals()['evolucao_pix'] = evolucao_pix

In [ ]:
# Análise Cash-In vs Cash-Out Últimos 3 Meses
print("📊 ANÁLISE CASH-IN vs CASH-OUT - ÚLTIMOS 3 MESES")
print("=" * 60)

# Verificar disponibilidade dos campos
campos_cash = ['cash_in', 'cash_out', 'cash_total']
campos_disponiveis = [c for c in campos_cash if c in df_base.columns]

print(f"Campos de cash disponíveis: {campos_disponiveis}")

if 'cash_in' in df_base.columns and 'cash_out' in df_base.columns:
    
    # Dados dos últimos 3 meses
    df_cash_3m = df_base[df_base['ano_mes'].isin(ultimos_3_meses)].copy()
    
    # Calcular diferença cash-in vs cash-out
    df_cash_3m['saldo_cash'] = df_cash_3m['cash_in'] - df_cash_3m['cash_out']
    df_cash_3m['cash_total_calc'] = df_cash_3m['cash_in'] + df_cash_3m['cash_out']
    
    print(f"\nANÁLISE POR PERÍODO:")
    print(f"{'Período':<10} {'Cash-In':<15} {'Cash-Out':<15} {'Saldo':<15} {'Total Mov':<15}")
    print("-" * 80)
    
    analise_cash_periodo = []
    
    for periodo in ultimos_3_meses:
        df_periodo = df_cash_3m[df_cash_3m['ano_mes'] == periodo]
        
        cash_in_total = df_periodo['cash_in'].sum()
        cash_out_total = df_periodo['cash_out'].sum()
        saldo_periodo = cash_in_total - cash_out_total
        movimentacao_total = cash_in_total + cash_out_total
        
        print(f"{periodo:<10} {cash_in_total:<15,.0f} {cash_out_total:<15,.0f} "
              f"{saldo_periodo:<15,.0f} {movimentacao_total:<15,.0f}")
        
        analise_cash_periodo.append({
            'periodo': periodo,
            'cash_in': cash_in_total,
            'cash_out': cash_out_total,
            'saldo': saldo_periodo,
            'movimentacao_total': movimentacao_total,
            'num_associados': len(df_periodo)
        })
    
    # Estatísticas gerais (último período)
    df_ultimo = df_cash_3m[df_cash_3m['ano_mes'] == ultimos_3_meses[-1]]
    
    print(f"\nESTATÍSTICAS ÚLTIMO PERÍODO ({ultimos_3_meses[-1]}):")
    print(f"  Total associados: {len(df_ultimo):,}")
    
    # Estatísticas cash-in
    cash_in_stats = df_ultimo['cash_in'].describe()
    print(f"\n  Cash-In:")
    print(f"    Média por associado: R$ {cash_in_stats['mean']:,.2f}")
    print(f"    Mediana: R$ {cash_in_stats['50%']:,.2f}")
    print(f"    Total: R$ {cash_in_stats['count'] * cash_in_stats['mean']:,.0f}")
    
    # Estatísticas cash-out
    cash_out_stats = df_ultimo['cash_out'].describe()
    print(f"\n  Cash-Out:")
    print(f"    Média por associado: R$ {cash_out_stats['mean']:,.2f}")
    print(f"    Mediana: R$ {cash_out_stats['50%']:,.2f}")
    print(f"    Total: R$ {cash_out_stats['count'] * cash_out_stats['mean']:,.0f}")
    
    # Saldo médio
    saldo_medio = df_ultimo['saldo_cash'].mean()
    saldo_mediano = df_ultimo['saldo_cash'].median()
    print(f"\n  Saldo (Cash-In - Cash-Out):")
    print(f"    Saldo médio: R$ {saldo_medio:,.2f}")
    print(f"    Saldo mediano: R$ {saldo_mediano:,.2f}")
    
    
    # Classificação por tipo de fluxo
    df_ultimo['tipo_fluxo'] = np.select([
        df_ultimo['saldo_cash'] > 100,
        df_ultimo['saldo_cash'] < -100,
        True
    ], [
        'Entrada_Liquida',
        'Saida_Liquida', 
        'Equilibrio'
    ])
    
    dist_fluxo = df_ultimo['tipo_fluxo'].value_counts()
    total_ultimo = len(df_ultimo)
    
    print(f"\nDISTRIBUIÇÃO POR TIPO DE FLUXO:")
    for tipo, qtd in dist_fluxo.items():
        perc = (qtd / total_ultimo) * 100
        print(f"  {tipo.replace('_', ' ')}: {qtd:,} ({perc:.1f}%)")
    
    # Análise por tipo de pessoa
    print(f"\nCASH FLOW POR TIPO DE PESSOA:")
    
    for pessoa in ['PF', 'PJ']:
        df_pessoa = df_ultimo[df_ultimo['tipo_pessoa'] == pessoa]
        if not df_pessoa.empty:
            cash_in_pessoa = df_pessoa['cash_in'].mean()
            cash_out_pessoa = df_pessoa['cash_out'].mean()
            saldo_pessoa = df_pessoa['saldo_cash'].mean()
            
            print(f"\n  {pessoa} (n={len(df_pessoa):,}):")
            print(f"    Cash-In médio: R$ {cash_in_pessoa:,.2f}")
            print(f"    Cash-Out médio: R$ {cash_out_pessoa:,.2f}")
            print(f"    Saldo médio: R$ {saldo_pessoa:,.2f}")
    
    # Top movimentadores
    print(f"\nTOP 10 MAIORES MOVIMENTAÇÕES TOTAIS:")
    top_movimentacao = df_ultimo.nlargest(10, 'cash_total_calc')
    
    for i, row in enumerate(top_movimentacao.itertuples(), 1):
        cpf_mask = str(row.cpf_cnpj)[-4:] if len(str(row.cpf_cnpj)) >= 4 else str(row.cpf_cnpj)
        print(f"  {i:2d}. CPF: ...{cpf_mask} | {row.tipo_pessoa} | "
              f"In: R${row.cash_in:,.0f} | Out: R${row.cash_out:,.0f} | "
              f"Saldo: R${row.saldo_cash:+,.0f}")
    
    # Correlação cash vs principalidade
    if 'pontos_principalidade' in df_ultimo.columns:
        print(f"\nCORRELAÇÃO CASH vs PRINCIPALIDADE:")
        
        corr_cash_in = df_ultimo['cash_in'].corr(df_ultimo['pontos_principalidade'])
        corr_cash_out = df_ultimo['cash_out'].corr(df_ultimo['pontos_principalidade'])
        corr_saldo = df_ultimo['saldo_cash'].corr(df_ultimo['pontos_principalidade'])
        
        print(f"  Cash-In vs Principalidade: {corr_cash_in:.3f}")
        print(f"  Cash-Out vs Principalidade: {corr_cash_out:.3f}")
        print(f"  Saldo vs Principalidade: {corr_saldo:.3f}")
    
    # Tendência de evolução
    if len(analise_cash_periodo) >= 2:
        print(f"\nTENDÊNCIA DE EVOLUÇÃO:")
        
        primeiro = analise_cash_periodo[0]
        ultimo = analise_cash_periodo[-1]
        
        var_cash_in = ((ultimo['cash_in'] / primeiro['cash_in']) - 1) * 100 if primeiro['cash_in'] > 0 else 0
        var_cash_out = ((ultimo['cash_out'] / primeiro['cash_out']) - 1) * 100 if primeiro['cash_out'] > 0 else 0
        var_movimentacao = ((ultimo['movimentacao_total'] / primeiro['movimentacao_total']) - 1) * 100 if primeiro['movimentacao_total'] > 0 else 0
        
        print(f"  Cash-In: {var_cash_in:+.1f}%")
        print(f"  Cash-Out: {var_cash_out:+.1f}%")
        print(f"  Movimentação Total: {var_movimentacao:+.1f}%")
    
    # Resumo cash
    resumo_cash = {
        'cash_in_total': analise_cash_periodo[-1]['cash_in'],
        'cash_out_total': analise_cash_periodo[-1]['cash_out'],
        'saldo_total': analise_cash_periodo[-1]['saldo'],
        'movimentacao_total': analise_cash_periodo[-1]['movimentacao_total'],
        'saldo_medio_associado': saldo_medio,
        'entrada_liquida': dist_fluxo.get('Entrada_Liquida', 0),
        'saida_liquida': dist_fluxo.get('Saida_Liquida', 0)
    }
    
    print(f"\n📋 RESUMO EXECUTIVO - CASH FLOW")
    print(f"Movimentação total: R$ {resumo_cash['movimentacao_total']:,.0f}")
    print(f"Saldo geral: R$ {resumo_cash['saldo_total']:+,.0f}")
    print(f"Entrada líquida: {resumo_cash['entrada_liquida']:,} associados")
    print(f"Saída líquida: {resumo_cash['saida_liquida']:,} associados")
    
    globals()['resumo_cash'] = resumo_cash
    globals()['analise_cash_periodo'] = analise_cash_periodo
    
else:
    print("⚠️  Campos de cash-in e cash-out não disponíveis para análise completa")
    resumo_cash = {'cash_in_total': 0, 'cash_out_total': 0, 'saldo_total': 0}

# Normalizando os campos booleanos para True/False
campos_bool = [
    'tem_conta_corrente', 'tem_poupanca', 'tem_investimento', 'tem_previdencia', 
    'tem_seguro', 'tem_consorcio', 'tem_cartao_credito', 'tem_credito', 
    'tem_cambio', 'tem_cobranca'
]

for col in campos_bool:
    if col in df_base.columns:
        df_base[col] = df_base[col].fillna(False).astype(bool)

# Convertendo colunas de data para datetime
for col in ['data_abertura', 'data_encerramento']:
    if col in df_base.columns:
        df_base[col] = pd.to_datetime(df_base[col], errors='coerce')

df_base.info()

## 9. Productos Faltantes: Categoría 1 y Categoría 2

Identificación de brechas de productos en dos categorías estratégicas y cálculo de cobertura.

In [ ]:
# Análise de Produtos Faltantes (Categoria 1 e 2)
print("📊 ANÁLISE DE PRODUTOS FALTANTES - CATEGORIAS 1 E 2")
print("=" * 70)

# Definição das categorias (ajustar conforme necessidade)
CATEGORIA_1 = {
    'PIX Ativo': 'cad_pix_ativo',
    'Cartão Débito': 'possui_cartao_debito',
    'App/Canais Digitais': 'transacao_app',
    'Débito Automático': 'debito_conta_ativo'
}

CATEGORIA_2 = {
    'Cartão Crédito': 'possui_cartao_credito',
    'Credenciamento': 'possui_adquirencia',
    'Cobrança': 'possui_cobranca',
    'Folha Pagamento': 'possui_folha_pagamento',
    'Domicílio': 'possui_domicilio',
    'Open Finance': 'ativou_open_finance'
}

ultimo_mes = df_base['ano_mes'].max()
df_prod_atual = df_base[df_base['ano_mes'] == ultimo_mes].copy()

# Garantir existência das colunas (criar se ausentes)
for col in list(CATEGORIA_1.values()) + list(CATEGORIA_2.values()):
    if col not in df_prod_atual.columns:
        df_prod_atual[col] = 0

# Calcular quantidade de produtos usados por categoria
cat1_cols = list(CATEGORIA_1.values())
cat2_cols = list(CATEGORIA_2.values())

df_prod_atual['cat1_usados'] = df_prod_atual[cat1_cols].sum(axis=1)
df_prod_atual['cat2_usados'] = df_prod_atual[cat2_cols].sum(axis=1)

df_prod_atual['cat1_faltantes'] = len(cat1_cols) - df_prod_atual['cat1_usados']
df_prod_atual['cat2_faltantes'] = len(cat2_cols) - df_prod_atual['cat2_usados']

# Cobertura (percentual de produtos utilizados em cada categoria)
df_prod_atual['cobertura_cat1'] = df_prod_atual['cat1_usados'] / len(cat1_cols)
df_prod_atual['cobertura_cat2'] = df_prod_atual['cat2_usados'] / len(cat2_cols)

# Distribuição de faltantes
print("\nDISTRIBUIÇÃO - CATEGORIA 1 (Produtos Transacionais Básicos):")
for falt in range(len(cat1_cols) + 1):
    qtd = (df_prod_atual['cat1_faltantes'] == falt).sum()
    perc = qtd / len(df_prod_atual) * 100
    print(f"  Faltam {falt} prod.: {qtd:,} ({perc:.1f}%)")

print("\nDISTRIBUIÇÃO - CATEGORIA 2 (Cross-Sell / Relacionamento):")
for falt in range(len(cat2_cols) + 1):
    qtd = (df_prod_atual['cat2_faltantes'] == falt).sum()
    perc = qtd / len(df_prod_atual) * 100
    print(f"  Faltam {falt} prod.: {qtd:,} ({perc:.1f}%)")

# Penetração por produto (ordenar)
print("\nPENETRAÇÃO POR PRODUTO (ÚLTIMO PERÍODO):")
penetracao = []
for nome, col in {**CATEGORIA_1, **CATEGORIA_2}.items():
    if col in df_prod_atual.columns:
        usuarios = df_prod_atual[col].sum()
        perc = usuarios / len(df_prod_atual) * 100 if len(df_prod_atual) > 0 else 0
        penetracao.append({'produto': nome, 'usuarios': int(usuarios), 'perc': perc})

df_penetracao = pd.DataFrame(penetracao).sort_values('perc', ascending=False)
for row in df_penetracao.itertuples():
    print(f"  {row.produto:<18} {row.usuarios:>8,} ({row.perc:5.1f}%)")

# Identificar oportunidades (associados com baixa cobertura transacional e alta principalidade)
if 'pontos_principalidade' in df_prod_atual.columns:
    mediana_princ = df_prod_atual['pontos_principalidade'].median()
    oportunidades_cat1 = df_prod_atual[(df_prod_atual['cobertura_cat1'] < 0.5) & (df_prod_atual['pontos_principalidade'] > mediana_princ)]
    oportunidades_cat2 = df_prod_atual[(df_prod_atual['cobertura_cat2'] < 0.5) & (df_prod_atual['pontos_principalidade'] > mediana_princ)]
    print(f"\nOPORTUNIDADES - Alta principalidade mas baixa cobertura transacional (<50%): {len(oportunidades_cat1):,}")
    print(f"OPORTUNIDADES - Alta principalidade mas baixa cobertura cross-sell (<50%): {len(oportunidades_cat2):,}")

# Resumo executivo
resumo_prod_faltantes = {
    'total': len(df_prod_atual),
    'cobertura_cat1_media': df_prod_atual['cobertura_cat1'].mean(),
    'cobertura_cat2_media': df_prod_atual['cobertura_cat2'].mean(),
    'sem_prod_cat1': (df_prod_atual['cat1_usados'] == 0).sum(),
    'sem_prod_cat2': (df_prod_atual['cat2_usados'] == 0).sum()
}

print("\n📋 RESUMO EXECUTIVO - PRODUTOS FALTANTES")
print(f"Cobertura média Cat 1: {resumo_prod_faltantes['cobertura_cat1_media']*100:,.1f}%")
print(f"Cobertura média Cat 2: {resumo_prod_faltantes['cobertura_cat2_media']*100:,.1f}%")
print(f"Sem nenhum produto Cat1: {resumo_prod_faltantes['sem_prod_cat1']:,} ({resumo_prod_faltantes['sem_prod_cat1']/len(df_prod_atual)*100:.1f}%)")
print(f"Sem nenhum produto Cat2: {resumo_prod_faltantes['sem_prod_cat2']:,} ({resumo_prod_faltantes['sem_prod_cat2']/len(df_prod_atual)*100:.1f}%)")

globals()['df_produtos_atual'] = df_prod_atual
globals()['df_penetracao_produtos'] = df_penetracao
globals()['resumo_prod_faltantes'] = resumo_prod_faltantes

## 10. SOW Sicredi vs Fora

Análise do indicador de Share of Wallet (SOW) para identificar potencial de captura adicional de relacionamento entre associados dentro e fora da cooperativa. Caso a métrica `sow` não esteja disponível na base, criamos um proxy a partir de pontos de principalidade e uso de produtos.

In [ ]:
# Análise SOW Sicredi vs Fora
print("📊 ANÁLISE SOW (SHARE OF WALLET) - PROXY")
print("=" * 60)

# Verificar existência ou criar proxy
ultimo_mes = df_base['ano_mes'].max()
df_sow = df_base[df_base['ano_mes'] == ultimo_mes].copy()

if 'sow' not in df_sow.columns:
    # Proxy simples: normalizar pontos de principalidade 0-1 e ponderar por cobertura de produtos transacionais
    produtos_trans = [c for c in ['cad_pix_ativo','transacao_app','possui_cartao_debito','debito_conta_ativo'] if c in df_sow.columns]
    if produtos_trans:
        df_sow['cobertura_trans'] = df_sow[produtos_trans].mean(axis=1)
    else:
        df_sow['cobertura_trans'] = 0
    princ_min = df_sow['pontos_principalidade'].min()
    princ_max = df_sow['pontos_principalidade'].max()
    escala = (df_sow['pontos_principalidade'] - princ_min) / (princ_max - princ_min) if princ_max > princ_min else 0
    df_sow['sow_proxy'] = (0.6 * escala + 0.4 * df_sow['cobertura_trans']).clip(0,1)
    sow_col = 'sow_proxy'
else:
    sow_col = 'sow'

# Faixas de SOW
bins = [0,0.25,0.5,0.75,1.01]
labels = ['Baixo','Médio-Baixo','Médio-Alto','Alto']
df_sow['faixa_sow'] = pd.cut(df_sow[sow_col], bins=bins, labels=labels, include_lowest=True)

# Distribuição geral
dist_sow = df_sow['faixa_sow'].value_counts().reindex(labels)
print("\nDISTRIBUIÇÃO GERAL SOW:")
for faixa, qtd in dist_sow.items():
    perc = qtd / len(df_sow) * 100 if len(df_sow) else 0
    print(f"  {faixa:<11}: {qtd:,} ({perc:.1f}%)")

# SOW por tipo de pessoa
print("\nSOW MÉDIO POR TIPO DE PESSOA:")
for tp in ['PF','PJ']:
    subset = df_sow[df_sow['tipo_pessoa']==tp]
    if not subset.empty:
        print(f"  {tp}: {subset[sow_col].mean():.3f} (n={len(subset):,})")

# Relação SOW x Principalidade
corr_sow_princ = df_sow[sow_col].corr(df_sow['pontos_principalidade']) if 'pontos_principalidade' in df_sow.columns else np.nan
print(f"\nCorrelação SOW vs Principalidade: {corr_sow_princ:.3f}")

# Oportunidades: alta principalidade (>=P75) e SOW baixo/medio-baixo
p75_princ = df_sow['pontos_principalidade'].quantile(0.75)
oportunidades_sow = df_sow[(df_sow['pontos_principalidade']>=p75_princ) & (df_sow['faixa_sow'].isin(['Baixo','Médio-Baixo']))]
print(f"\nOportunidades de expansão (alta principalidade & SOW<=0.5): {len(oportunidades_sow):,}")

# Resumo executivo
resumo_sow = {
    'total': len(df_sow),
    'sow_medio': df_sow[sow_col].mean(),
    'alta_participacao': (df_sow['faixa_sow']=='Alto').sum(),
    'baixo_share': (df_sow['faixa_sow']=='Baixo').sum(),
    'corr_princ': corr_sow_princ,
    'oportunidades': len(oportunidades_sow)
}

print("\n📋 RESUMO EXECUTIVO - SOW")
print(f"SOW médio: {resumo_sow['sow_medio']:.3f}")
print(f"Alto SOW: {resumo_sow['alta_participacao']:,} ({resumo_sow['alta_participacao']/len(df_sow)*100:.1f}%)")
print(f"Baixo SOW: {resumo_sow['baixo_share']:,} ({resumo_sow['baixo_share']/len(df_sow)*100:.1f}%)")
print(f"Correlação com Principalidade: {resumo_sow['corr_princ']:.3f}")
print(f"Oportunidades expansão: {resumo_sow['oportunidades']:,}")

globals()['resumo_sow'] = resumo_sow
globals()['df_sow'] = df_sow

## 11. Análise PJ: Domicílio, Cobrança, PIX e Uso Transacional

Foco em produtos críticos para relacionamento PJ: domicílio, cobrança, credenciamento, folha pagamento e adoção de PIX. Avaliação de penetração, gaps e relação com principalidade.

In [ ]:
# Análise PJ detalhada
print("📊 ANÁLISE PJ - PRODUTOS E USO TRANSACIONAL")
print("=" * 65)

ultimo_mes = df_base['ano_mes'].max()
df_pj = df_base[(df_base['ano_mes']==ultimo_mes) & (df_base['tipo_pessoa']=='PJ')].copy()
print(f"Total PJ analisados: {len(df_pj):,}")

produtos_pj = {
    'Domicílio': 'possui_domicilio',
    'Cobrança': 'possui_cobranca',
    'Credenciamento': 'possui_adquirencia',
    'Folha Pagamento': 'possui_folha_pagamento',
    'PIX Ativo': 'cad_pix_ativo'
}

# Garantir colunas
for c in produtos_pj.values():
    if c not in df_pj.columns:
        df_pj[c] = 0

# Penetração
penetracao_pj = []
for nome, col in produtos_pj.items():
    usuarios = df_pj[col].sum()
    perc = usuarios/len(df_pj)*100 if len(df_pj)>0 else 0
    penetracao_pj.append({'produto': nome, 'usuarios': int(usuarios), 'perc': perc})

df_pen_pj = pd.DataFrame(penetracao_pj).sort_values('perc', ascending=False)
print("\nPENETRAÇÃO DE PRODUTOS PJ:")
for r in df_pen_pj.itertuples():
    print(f"  {r.produto:<16} {r.usuarios:>7,} ({r.perc:5.1f}%)")

# Cobertura composta (quantos desses chave usa)
cols_chave = list(produtos_pj.values())
df_pj['prod_chave_usados'] = df_pj[cols_chave].sum(axis=1)
print("\nDistribuição nº de produtos chave usados:")
for k in range(len(cols_chave)+1):
    qtd = (df_pj['prod_chave_usados']==k).sum()
    print(f"  Usa {k} prod.: {qtd:,} ({qtd/len(df_pj)*100:.1f}%)")

# PIX transações (intensidade) se existir
if 'pix_trans_30d' in df_pj.columns and 'cad_pix_ativo' in df_pj.columns:
    ativos_pix = df_pj[df_pj['cad_pix_ativo']==1]
    if not ativos_pix.empty:
        ativos_pix['intensidade_pix'] = pd.cut(
            ativos_pix['pix_trans_30d'],
            bins=[0,1,5,15,float('inf')],
            labels=['Muito_Baixo','Baixo','Médio','Alto']
        )
        print("\nINTENSIDADE PIX (PJ com PIX Ativo):")
        dist_int = ativos_pix['intensidade_pix'].value_counts()
        for cat, qtd in dist_int.items():
            print(f"  {cat:<12}: {qtd:,} ({qtd/len(ativos_pix)*100:.1f}%)")

# Relação cada produto x principalidade média
if 'pontos_principalidade' in df_pj.columns:
    print("\nPRINCIPALIDADE MÉDIA POR ADOÇÃO DE PRODUTO:")
    princ_med_base = df_pj['pontos_principalidade'].mean()
    for nome, col in produtos_pj.items():
        if col in df_pj.columns:
            media_com = df_pj[df_pj[col]==1]['pontos_principalidade'].mean()
            media_sem = df_pj[df_pj[col]==0]['pontos_principalidade'].mean()
            uplift = media_com - media_sem
            print(f"  {nome:<16} Com: {media_com:7.2f} | Sem: {media_sem:7.2f} | Uplift: {uplift:+.2f}")

# Oportunidades: alta principalidade mas sem domicílio ou cobrança
if 'pontos_principalidade' in df_pj.columns:
    p75 = df_pj['pontos_principalidade'].quantile(0.75)
    oportunidades_domicilio = df_pj[(df_pj['pontos_principalidade']>=p75) & (df_pj['possui_domicilio']==0)]
    oportunidades_cobranca = df_pj[(df_pj['pontos_principalidade']>=p75) & (df_pj['possui_cobranca']==0)]
    print(f"\nOportunidades Domicílio (alto princ, sem domicílio): {len(oportunidades_domicilio):,}")
    print(f"Oportunidades Cobrança (alto princ, sem cobrança): {len(oportunidades_cobranca):,}")

resumo_pj = {
    'total_pj': len(df_pj),
    'pix_ativo': df_pj['cad_pix_ativo'].sum() if 'cad_pix_ativo' in df_pj.columns else 0,
    'domicilio': df_pj['possui_domicilio'].sum() if 'possui_domicilio' in df_pj.columns else 0,
    'cobranca': df_pj['possui_cobranca'].sum() if 'possui_cobranca' in df_pj.columns else 0
}
print("\n📋 RESUMO EXECUTIVO - PJ")
print(f"Total PJ: {resumo_pj['total_pj']:,}")
print(f"Domicílio: {resumo_pj['domicilio']:,} ({resumo_pj['domicilio']/len(df_pj)*100:.1f}%)")
print(f"Cobrança: {resumo_pj['cobranca']:,} ({resumo_pj['cobranca']/len(df_pj)*100:.1f}%)")
print(f"PIX Ativo: {resumo_pj['pix_ativo']:,} ({resumo_pj['pix_ativo']/len(df_pj)*100:.1f}%)")

globals()['resumo_pj'] = resumo_pj
globals()['df_pj_analise'] = df_pj

## 12. Uso por Faixa de Dias sem Movimento

Avalia adoção de produtos, principalidade e SOW conforme faixas de inatividade (dias sem movimento).

In [ ]:
# Uso por faixa de dias sem movimento
print("📊 ANÁLISE POR FAIXA DE INATIVIDADE")
print("=" * 50)

ultimo_mes = df_base['ano_mes'].max()
df_mov = df_base[df_base['ano_mes']==ultimo_mes].copy()

# Cálculo direto de dias_sem_mov usando 'ult_movimento' (prioritário)
if 'ult_movimento' in df_mov.columns:
    df_mov['ult_movimento'] = pd.to_datetime(df_mov['ult_movimento'], errors='coerce')
    hoje = pd.Timestamp.today().normalize()
    df_mov['dias_sem_mov'] = (hoje - df_mov['ult_movimento']).dt.days
else:
    # Fallback legacy (mantido como contingência)
    possiveis_col_datas = [
        'ult_movimento_dt', 'dt_ult_movimento', 'data_ult_movimento',
        'dt_ult_movto', 'dt_ult_mov', 'dat_ult_movimento'
    ]
    col_dt_encontrada = next((c for c in possiveis_col_datas if c in df_mov.columns), None)
    if col_dt_encontrada:
        df_mov[col_dt_encontrada] = pd.to_datetime(df_mov[col_dt_encontrada], errors='coerce')
        hoje = pd.Timestamp.today().normalize()
        df_mov['dias_sem_mov'] = (hoje - df_mov[col_dt_encontrada]).dt.days

# Criar faixa_movimento (somente se dias_sem_mov disponível)
if 'dias_sem_mov' in df_mov.columns:
    df_mov['faixa_movimento'] = pd.cut(
        df_mov['dias_sem_mov'],
        bins=[-1,20,45,float('inf')],
        labels=['Menos_20_dias','20_45_dias','Mais_45_dias']
    )

faixas = ['Menos_20_dias','20_45_dias','Mais_45_dias']
produtos_chave = ['cad_pix_ativo','transacao_app','possui_cartao_debito','possui_cartao_credito']

print("\nMÉTRICAS POR FAIXA:")
print(f"{'Faixa':<15} {'Qtde':>8} {'Princ Méd':>10} {'PIX%':>7} {'App%':>7} {'Déb%':>7} {'Créd%':>7}")
print('-'*70)
resumo_faixas = []

if 'faixa_movimento' in df_mov.columns:
    for f in faixas:
        subset = df_mov[df_mov['faixa_movimento'] == f]
        if subset.empty: 
            continue
        princ_med = subset['pontos_principalidade'].mean() if 'pontos_principalidade' in subset.columns else float('nan')
        linha = {'faixa': f, 'qtde': len(subset), 'princ_media': princ_med}
        valores_print = [f"{f:<15} {len(subset):>8,} {princ_med:>10.1f}"]
        for prod in produtos_chave:
            if prod in subset.columns:
                perc = subset[prod].mean()*100
                linha[prod+'_perc'] = perc
                valores_print.append(f"{perc:>6.1f}%")
            else:
                valores_print.append(f"{'-':>6}")
        print(' '.join(valores_print))
        resumo_faixas.append(linha)
else:
    print("⚠️  Não foi possível criar 'faixa_movimento' (campo 'ult_movimento' ausente ou inválido).")

# Correlação dias vs principalidade
if {'dias_sem_mov','pontos_principalidade'} <= set(df_mov.columns):
    corr_mov_princ = df_mov['dias_sem_mov'].corr(df_mov['pontos_principalidade'])
    print(f"\nCorrelação Dias Sem Mov. vs Principalidade: {corr_mov_princ:.3f}")

# Oportunidades: inativos + alta principalidade + sem PIX
if {'dias_sem_mov','pontos_principalidade','cad_pix_ativo'} <= set(df_mov.columns):
    p75 = df_mov['pontos_principalidade'].quantile(0.75)
    oportunidades_inativos = df_mov[
        (df_mov['dias_sem_mov']>45) &
        (df_mov['pontos_principalidade']>=p75) &
        (df_mov['cad_pix_ativo']==0)
    ]
    print(f"Inativos com alta principalidade sem PIX: {len(oportunidades_inativos):,}")

# Resumo
if 'faixa_movimento' in df_mov.columns:
    resumo_movimento = {
        'total': len(df_mov),
        'menor_20': (df_mov['faixa_movimento']=='Menos_20_dias').sum(),
        '20_45': (df_mov['faixa_movimento']=='20_45_dias').sum(),
        'mais_45': (df_mov['faixa_movimento']=='Mais_45_dias').sum()
    }
else:
    resumo_movimento = {'total': len(df_mov), 'menor_20': 0, '20_45': 0, 'mais_45': 0}

print("\n📋 RESUMO EXECUTIVO - FAIXAS INATIVIDADE")
print(f"Menos 20 dias: {resumo_movimento['menor_20']:,}")
print(f"20-45 dias: {resumo_movimento['20_45']:,}")
print(f"Mais 45 dias: {resumo_movimento['mais_45']:,}")

globals()['resumo_movimento'] = resumo_movimento
globals()['resumo_faixas_movimento'] = resumo_faixas

## 13. Visualizações e Gráficos Comparativos

Gráficos para apresentação: distribuições, evoluções, heatmaps de correlação e penetração de produtos.

In [ ]:
# Visualizações chave para apresentação
print("🎨 GERAÇÃO DE GRÁFICOS")

import plotly.express as px
import plotly.graph_objects as go

figs_apresentacao = {}

# 1. Distribuição Principalidade (último mês)
ultimo_mes = df_base['ano_mes'].max()
df_ult = df_base[df_base['ano_mes']==ultimo_mes]
if 'pontos_principalidade' in df_ult.columns:
    fig = px.histogram(df_ult, x='pontos_principalidade', nbins=40, title='Distribuição de Principalidade')
    figs_apresentacao['dist_principalidade'] = fig
    fig.show()

# 2. Evolução média principalidade 3 últimos meses
if 'pontos_principalidade' in df_base.columns:
    evol_princ = df_base[df_base['ano_mes'].isin(ultimos_3_meses)].groupby('ano_mes')['pontos_principalidade'].mean().reset_index()
    fig = px.line(evol_princ, x='ano_mes', y='pontos_principalidade', markers=True, title='Evolução Média Principalidade (3M)')
    figs_apresentacao['evol_principalidade'] = fig
    fig.show()

# 3. Penetração produtos (bar)
if 'df_penetracao_produtos' in globals():
    fig = px.bar(df_penetracao_produtos.sort_values('perc'), x='perc', y='produto', orientation='h', title='Penetração de Produtos (%)')
    figs_apresentacao['penet_produtos'] = fig
    fig.show()

# 4. Heatmap correlação (subset numérico)
num_cols = []
for c in ['pontos_principalidade','isa_analise','cad_pix_ativo','possui_cartao_credito','cash_in','cash_out']:
    if c in df_ult.columns:
        num_cols.append(c)
if len(num_cols) >= 3:
    corr = df_ult[num_cols].corr()
    fig = px.imshow(corr, text_auto='.2f', title='Heatmap Correlação Indicadores')
    figs_apresentacao['heatmap_corr'] = fig
    fig.show()

# 5. SOW Faixa x Principalidade média
if 'df_sow' in globals() and 'faixa_sow' in df_sow.columns:
    base_sow = df_sow.groupby('faixa_sow')['pontos_principalidade'].mean().reset_index()
    fig = px.bar(base_sow, x='faixa_sow', y='pontos_principalidade', title='Principalidade Média por Faixa SOW')
    figs_apresentacao['princ_por_sow'] = fig
    fig.show()

# 6. Inatividade x Adoção PIX
if 'faixa_movimento' in df_ult.columns and 'cad_pix_ativo' in df_ult.columns:
    inat_pix = df_ult.groupby('faixa_movimento')['cad_pix_ativo'].mean().reset_index()
    fig = px.bar(inat_pix, x='faixa_movimento', y='cad_pix_ativo', title='Penetração PIX por Faixa de Inatividade', labels={'cad_pix_ativo':'PIX %'})
    figs_apresentacao['pix_inatividade'] = fig
    fig.show()

print("\nTotal de gráficos gerados:", len(figs_apresentacao))
globals()['figs_apresentacao'] = figs_apresentacao

## 14. Modelo de Pré-Churn (Machine Learning)

Criação de features, definição de variável alvo (proxy churn = queda principalidade + inatividade + ausência de PIX), treino de modelo e identificação de associados com maior propensão a churn.

In [ ]:
# Modelo de Pré-Churn
print("🤖 MODELO PRÉ-CHURN - EXPERIMENTO INICIAL")
print("="*55)

# Preparar base histórica mínima (últimos 4 registros por associado já carregados na query inicial)
base_ml = df_base.copy()

# Criar alvo churn_proxy: queda recente + inatividade + sem PIX
# Condições: (queda_flag==1 no último mês) OU (soma_quedas>=2) E (dias_sem_mov>45) E (cad_pix_ativo==0)
ultimo_mes = base_ml['ano_mes'].max()
ult = base_ml[base_ml['ano_mes']==ultimo_mes].copy()

# Garantir colunas
for c in ['queda_flag','soma_quedas','dias_sem_mov','cad_pix_ativo']:
    if c not in ult.columns:
        ult[c] = 0

ult['churn_proxy'] = np.where(
    ( (ult['queda_flag']==1) | (ult['soma_quedas']>=2) ) &
    (ult['dias_sem_mov']>45) &
    (ult['cad_pix_ativo']==0),
    1,0
)

print(f"Taxa alvo churn proxy: {ult['churn_proxy'].mean()*100:.2f}%")

# Features selecionadas
features = [c for c in [
    'pontos_principalidade','cad_pix_ativo','transacao_app','possui_cartao_debito',
    'possui_cartao_credito','possui_adquirencia','possui_cobranca','possui_domicilio',
    'possui_folha_pagamento','ativou_open_finance','dias_sem_mov','cash_in','cash_out'
] if c in ult.columns]

df_model = ult[['cpf_cnpj','churn_proxy'] + features].dropna().copy()

if df_model['churn_proxy'].nunique() >= 2 and len(df_model) > 100:
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score, classification_report
    from sklearn.ensemble import RandomForestClassifier

    X = df_model[features]
    y = df_model['churn_proxy']

    # Balance simples se a taxa for muito baixa
    if y.mean() < 0.05:
        # Oversampling simples (duplicar positivos)
        pos = df_model[df_model['churn_proxy']==1]
        mult = int((len(df_model)-len(pos))/len(pos)*0.5)+1 if len(pos)>0 else 1
        df_model_bal = pd.concat([df_model, pd.concat([pos]*mult)])
        X = df_model_bal[features]
        y = df_model_bal['churn_proxy']
        print(f"Aplicado oversampling simples. Nova taxa: {y.mean()*100:.2f}%")

    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3, random_state=42, stratify=y)

    rf = RandomForestClassifier(n_estimators=300, random_state=42, class_weight='balanced_subsample')
    rf.fit(X_train,y_train)
    probas = rf.predict_proba(X_test)[:,1]
    preds = (probas>=0.5).astype(int)

    auc = roc_auc_score(y_test, probas)
    print(f"ROC AUC: {auc:.3f}")
    print("\nRelatório classificação:")
    print(classification_report(y_test,preds, digits=3))

    # Importância features
    imp = pd.DataFrame({'feature':features,'importance':rf.feature_importances_}).sort_values('importance', ascending=False)
    print("\nIMPORTÂNCIA DAS FEATURES:")
    for r in imp.itertuples():
        print(f"  {r.feature:<25} {r.importance:6.3f}")

    # Top risco (aplicar em todo dataset atual)
    full_probas = rf.predict_proba(ult[features])[:,1]
    ult['prob_churn'] = full_probas
    top_risco = ult.nlargest(20,'prob_churn')[['cpf_cnpj','prob_churn','pontos_principalidade','dias_sem_mov']]
    print("\nTOP 20 ASSOCIADOS MAIOR RISCO (proxy):")
    for i,row in enumerate(top_risco.itertuples(),1):
        print(f"  {i:2d}. ...{str(row.cpf_cnpj)[-4:]} | Prob: {row.prob_churn:.2f} | Princ: {row.pontos_principalidade:.1f} | Dias SEM: {row.dias_sem_mov}")

    globals()['modelo_churn'] = rf
    globals()['importancia_churn'] = imp
    globals()['scoring_churn'] = ult[['cpf_cnpj','prob_churn','churn_proxy']]
else:
    print("⚠️ Base insuficiente ou alvo sem variabilidade para modelagem.")

## 15. Geração de Relatório Final

Conversão dos principais resumos, tabelas e métricas em um relatório executável (HTML/PDF) para apresentação. Inclui apêndice técnico e recomendações estratégicas.

In [ ]:
# Geração de Relatório Final (HTML simplificado)
print("📝 GERAÇÃO DE RELATÓRIO FINAL")

from pathlib import Path

resumos = {}
for nome_var in [
    'resumo_contas','resumo_bbm','resumo_evolucao','resumo_produtos','resumo_isa','resumo_pix',
    'resumo_cash','resumo_prod_faltantes','resumo_sow','resumo_pj','resumo_movimento'
]:
    if nome_var in globals():
        resumos[nome_var] = globals()[nome_var]

html_parts = ["<html><head><meta charset='utf-8'><title>Relatório Principalidade</title>" \
              "<style>body{font-family:Arial; margin:30px;} h2{border-bottom:1px solid #ccc;} table{border-collapse:collapse;} td,th{border:1px solid #ddd;padding:4px 8px;} .kpi{display:inline-block;margin:10px 20px;}</style></head><body>"]
html_parts.append("<h1>Relatório Executivo - Principalidade</h1>")
html_parts.append(f"<p>Data geração: {datetime.now().strftime('%d/%m/%Y %H:%M')}</p>")

# KPIs principais (se existirem)
if 'resumo_evolucao' in resumos and 'resumo_pix' in resumos:
    kpis = [
        ("Associados Analisados", resumos['resumo_evolucao'].get('total_analisados','-')),
        ("% Subiram", f"{resumos['resumo_evolucao'].get('subiu',0)/resumos['resumo_evolucao'].get('total_analisados',1)*100:.1f}%"),
        ("PIX Penetração", f"{resumos['resumo_pix'].get('perc_penetracao',0):.1f}%"),
        ("ISA Médio", f"{resumos.get('resumo_isa',{}).get('isa_medio',0):.2f}"),
        ("SOW Médio", f"{resumos.get('resumo_sow',{}).get('sow_medio',0):.2f}"),
    ]
    html_parts.append("<div>")
    for k,v in kpis:
        html_parts.append(f"<div class='kpi'><strong>{k}</strong><br>{v}</div>")
    html_parts.append("</div>")

# Tabelas de resumos
for nome, dados in resumos.items():
    html_parts.append(f"<h2>{nome.replace('_',' ').title()}</h2>")
    html_parts.append("<table><tr><th>Métrica</th><th>Valor</th></tr>")
    for k,v in dados.items():
        html_parts.append(f"<tr><td>{k}</td><td>{v}</td></tr>")
    html_parts.append("</table>")

html_parts.append("<h2>Recomendações Estratégicas</h2>")
html_parts.append("<ul>")
html_parts.append("<li>Acelerar ativação PIX em faixas de alta principalidade e inatividade prolongada.</li>")
html_parts.append("<li>Priorizar cross-sell em contas com SOW baixo e principalidade elevada.</li>")
html_parts.append("<li>Atacar gaps de produtos transacionais básicos para elevar retenção.</li>")
html_parts.append("<li>Focar PJ sem domicílio/cobrança mas com relacionamento forte.</li>")
html_parts.append("<li>Acompanhar cohort de queda sucessiva (3 quedas) para prevenção churn.</li>")
html_parts.append("</ul>")

html_parts.append("<p><em>Relatório gerado automaticamente. Ajustes visuais podem ser aplicados em ferramenta de apresentação.</em></p>")
html_parts.append("</body></html>")

html_final = ''.join(html_parts)
saida = Path('relatorio_principalidade.html')
saida.write_text(html_final, encoding='utf-8')
print(f"Relatório salvo em: {saida.resolve()}")

globals()['relatorio_html'] = html_final

In [ ]:
# Validações finais e export opcional
from IPython.display import display

print('=== Validações Finais ===')
if 'df_base' not in globals():
    raise RuntimeError('df_base inexistente.')

print(f"Registros df_base: {len(df_base):,}")
print(f"Colunas ({len(df_base.columns)}): {list(df_base.columns)[:25]} ...")

# 1. Checar duplicidade da chave principal (cpf_cnpj + ano_mes se existir)
if 'cpf_cnpj' in df_base.columns and 'ano_mes' in df_base.columns:
    dup = df_base.duplicated(subset=['cpf_cnpj','ano_mes']).sum()
    print(f"Duplicados cpf_cnpj+ano_mes: {dup}")
else:
    print('Aviso: Não foi possível checar duplicidade (faltam colunas).')

# 2. Percentual de tipo_pessoa preenchido
if 'tipo_pessoa' in df_base.columns:
    perc = df_base['tipo_pessoa'].notna().mean()*100
    print(f"tipo_pessoa preenchido: {perc:0.2f}%")

# 3. Amostra final
print('\nAmostra final:')
cols_show = [c for c in ['cpf_cnpj','ano_mes','pontos_principalidade','tipo_pessoa','segmento'] if c in df_base.columns]
print(df_base[cols_show].head())

# 4. Export opcional
EXPORTAR = False  # coloque True se quiser exportar
CAMINHO_EXPORT = os.path.join(RUTAS['saida_dir'], 'principalidade_merge_bruto.parquet')
if EXPORTAR:
    df_base.to_parquet(CAMINHO_EXPORT, index=False)
    print(f"✓ Exportado para {CAMINHO_EXPORT}")
else:
    print('Exportação desativada (EXPORTAR=False).')

# Conclusión y Validaciones Finales

Se realizaron las cargas brutas, merges controlados y análisis inicial sin ninguna normalización automática de identificadores ni inferencia de tipo de persona.

Próximos passos (opcionales):
- Ajustar manualmente `cpf_cnpj` activando la celda opcional (solo si lo necesitas).
- Agregar análisis adicionales en nuevas celdas sin modificar `df_base` original.
- Exportar subconjuntos específicos si deseas trabajar en otro entorno.

La siguiente celda ejecuta validaciones finales e opcionalmente guarda un parquet limpio. Ajusta rutas según necesidad.